In [2]:
# Import libraries

import pandas as pd
import numpy as np
import os
import random
from ortools.graph import pywrapgraph
import xlsxwriter
import xlwt

In [3]:
# Get directory location

os.getcwd()

'/Users/jwallin/Desktop/OpE-TechSquad'

In [4]:
#Import each sheet in the 'IN_Supply_Chain_Model.xlsx' file as its own dataframe

df_nodes = pd.read_excel('IN_Supply_Chain_Model.xlsx', sheet_name = 'Nodes', engine='openpyxl')
df_edges = pd.read_excel('IN_Supply_Chain_Model.xlsx', sheet_name = 'Edges', engine='openpyxl')
df_risks_location = pd.read_excel('IN_Supply_Chain_Model.xlsx', sheet_name = 'Risks - Location', engine='openpyxl')
df_risks_type = pd.read_excel('IN_Supply_Chain_Model.xlsx', sheet_name = 'Risks - Type', engine='openpyxl')
df_risks_list = pd.read_excel('IN_Supply_Chain_Model.xlsx', sheet_name = 'Risks - List', engine='openpyxl')

In [5]:
# Make copies of each df for use in excel export later in code

df_nodes_orig = df_nodes.copy()
df_risks_location_orig = df_risks_location.copy()
df_risks_type_orig = df_risks_type.copy()
df_risks_list_orig = df_risks_list.copy()

In [6]:
# Create df_nodes_gephi from df_nodes to format for csv export and gephi import

df_nodes_gephi = df_nodes.copy()
df_nodes_gephi = df_nodes_gephi[['Node ID', 'Node Name', 'Latitude', 'Longitude']]
df_nodes_gephi = df_nodes_gephi.rename(columns = {'Node ID': 'Id', 'Node Name': 'Label'})
# df_nodes_gephi

In [7]:
# Create random latitude and longitude for df_nodes_gephi_adj to be able to post online

df_nodes_gephi_adj = df_nodes_gephi.copy()
for i in range(len(df_nodes_gephi_adj)):
    df_nodes_gephi_adj.loc[i, 'Latitude'] += random.uniform(-1, 1)
    df_nodes_gephi_adj.loc[i, 'Longitude'] += random.uniform(-1, 1)
#df_nodes_gephi_adj

In [8]:
# Create df_edges_gephi from df_edges to format for csv export and gephi import

df_edges_gephi = df_edges.copy()
df_edges_gephi = df_edges_gephi[['Start Node', 'End Node', 'Capacity']]
df_edges_gephi = df_edges_gephi.rename(columns = {'Start Node': 'Source', 'End Node': 'Target',
                                                 'Capacity': 'Weight'})
df_edges_gephi['Type'] = 'Directed'
#df_edges_gephi

In [9]:
# Create variable 'nodes_len' to use throughout code

nodes_len = len(df_nodes)
nodes_len

49

In [10]:
# Assign the 'risks - type' value to each node

df_comb1 = pd.merge(df_nodes, df_risks_type, how = 'left', on = 'Type')
df_comb1['Expected_Impact'] = df_comb1['Probability'] * df_comb1['Impact']
df_comb1 = df_comb1[['Node ID', 'Expected_Impact']]
df_comb1 = df_comb1.groupby(['Node ID']).sum()
#df_comb1

In [11]:
# Add the 'risks - type' values to df_nodes

df_nodes = pd.merge(df_nodes, df_comb1, how = 'left', on = 'Node ID')
df_nodes = df_nodes.rename(columns = {'Expected_Impact': 'Risks - Type'})
df_nodes

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,Latitude,Longitude,Risks - Type
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,47.280,-122.20,0.12
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,45.660,-122.53,0.06
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,45.800,-118.53,0.06
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,47.160,-116.56,0.06
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,45.210,-107.77,0.06
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,47.810,-103.28,0.12
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,41.380,-109.79,0.06
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,42.680,-112.16,0.12
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,39.840,-112.25,0.06
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,40.710,-117.00,0.06


In [12]:
# Add column 'Node & ID' that combines the word 'Node' with the node ID in order to merge with df_risks_list

df_nodes['Node & ID'] = 'Node ' + df_nodes['Node ID'].astype(str)
df_nodes

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,Latitude,Longitude,Risks - Type,Node & ID
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,47.280,-122.20,0.12,Node 1
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,45.660,-122.53,0.06,Node 2
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,45.800,-118.53,0.06,Node 3
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,47.160,-116.56,0.06,Node 4
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,45.210,-107.77,0.06,Node 5
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,47.810,-103.28,0.12,Node 6
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,41.380,-109.79,0.06,Node 7
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,42.680,-112.16,0.12,Node 8
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,39.840,-112.25,0.06,Node 9
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,40.710,-117.00,0.06,Node 10


In [13]:
# Add column 'Node/Edge & ID' that combines values from columns 'Edge or Node' and 'Node ID' to merge with df_nodes

df_risks_list['Node/Edge & ID'] = df_risks_list['Edge or Node'] + ' ' + df_risks_list['ID'].astype(str)
df_risks_list

,Risk Name,Description,Type,Edge or Node,ID,Probability,Impact,Node/Edge & ID
0,Maple syrup flood,Canadian maple syrup flood moving South,Accident,Node,6,0.010,0.10,Node 6
1,Lobster blocade,Lobsters blocking ship transport,Transport,Edge,98,0.050,0.25,Edge 98
2,Nails on highway,Nails pop truck tires,Transport,Edge,44,0.005,0.30,Edge 44


In [14]:
# Assign the 'risks - list' value to each node

df_comb1 = pd.merge(df_nodes, df_risks_list, how = 'left', left_on = 'Node & ID', right_on = 'Node/Edge & ID')
df_comb1 = df_comb1.fillna(0)
df_comb1['Expected_Impact'] = df_comb1['Probability'] * df_comb1['Impact']
df_comb1 = df_comb1[['Node ID', 'Expected_Impact']]
df_comb1 = df_comb1.groupby(['Node ID']).sum()
df_comb1

,Expected_Impact
Node ID,
1,0.000
2,0.000
3,0.000
4,0.000
5,0.000
6,0.001
7,0.000
8,0.000
9,0.000


In [15]:
# Add the 'risks - list' values to df_nodes

df_nodes = pd.merge(df_nodes, df_comb1, how = 'left', on = 'Node ID')
df_nodes = df_nodes.rename(columns = {'Expected_Impact': 'Risks - List'})
df_nodes.pop('Node & ID')
df_nodes

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,Latitude,Longitude,Risks - Type,Risks - List
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,47.280,-122.20,0.12,0.000
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,45.660,-122.53,0.06,0.000
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,45.800,-118.53,0.06,0.000
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,47.160,-116.56,0.06,0.000
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,45.210,-107.77,0.06,0.000
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,47.810,-103.28,0.12,0.001
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,41.380,-109.79,0.06,0.000
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,42.680,-112.16,0.12,0.000
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,39.840,-112.25,0.06,0.000
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,40.710,-117.00,0.06,0.000


In [16]:
# Add column 'Risk ID' to df_risks_location that gives a unique name to each risk

df_risks_location['Risk ID'] = df_risks_location['Box or Circle'] + '_' + df_risks_location.index.astype(str)
df_risks_location

,Risk Name,Description,Type,Box or Circle,Circle Lat,Circle Lon,Inner Distance,Outer Distance,Box 1 Lat,Box 1 Lon,Box 2 Lat,Box 2 Lon,Probability,Impact,Risk ID
0,Hurricane,Gulf Coast hurricane,Natural Factors,Circle,28.964908,-92.736005,0.0,400.0,NaN,NaN,NaN,NaN,0.40,0.65,Circle_0
1,Polar vortex,Extreme cold weather,Natural Factors,Box,NaN,NaN,NaN,NaN,46.803830,-94.713544,41.512702,-78.277998,0.10,0.30,Box_1
2,Heat wave,Extreme hot weather,Natural Factors,Box,NaN,NaN,NaN,NaN,35.357709,-121.476239,30.073861,-102.272138,0.05,0.20,Box_2


In [17]:
# Create lists for each risk in df_risks_location

gbl = globals()

threatlist = df_risks_location['Risk ID'].tolist()
for x in threatlist:
    gbl[x] = []
    
threatlist

['Circle_0', 'Box_1', 'Box_2']

In [18]:
# Calculating the risk from each risk in df_risks_location and attaching to each node in df_nodes

for x in range(len(df_nodes)):
    for y, z in zip(range(len(df_risks_location)), threatlist):
        if  df_risks_location.loc[y, 'Box or Circle'] == 'Box':
            if ((df_nodes.loc[x, 'Latitude'] <=
                max(df_risks_location.loc[y, 'Box 1 Lat'], df_risks_location.loc[y, 'Box 2 Lat'])) &
                   (df_nodes.loc[x, 'Latitude'] >=
                min(df_risks_location.loc[y, 'Box 1 Lat'], df_risks_location.loc[y, 'Box 2 Lat'])) &
                    (df_nodes.loc[x, 'Longitude'] <=
                max(df_risks_location.loc[y, 'Box 1 Lon'], df_risks_location.loc[y, 'Box 2 Lon'])) &
                    (df_nodes.loc[x, 'Longitude'] >=
                min(df_risks_location.loc[y, 'Box 1 Lon'], df_risks_location.loc[y, 'Box 2 Lon']))):
                    gbl[z].append(df_risks_location.loc[y, 'Probability'] * df_risks_location.loc[y, 'Impact'])
            else:
                gbl[z].append(0)
        else:
            interva = (6371 * np.arccos(np.cos(np.radians(90 - df_nodes.loc[x, 'Latitude'])) *
                np.cos(np.radians(90 - df_risks_location.loc[y, 'Circle Lat'])) +
                np.sin(np.radians(90- df_nodes.loc[x, 'Latitude'])) *
                np.sin(np.radians(90 - df_risks_location.loc[y, 'Circle Lat'])) *
                np.cos(np.radians(df_nodes.loc[x, 'Longitude'] - df_risks_location.loc[y, 'Circle Lon']))))
            if ((interva >= df_risks_location.loc[y, 'Inner Distance']) & 
                (interva <= df_risks_location.loc[y, 'Outer Distance'])): 
                    gbl[z].append(df_risks_location.loc[y, 'Probability'] * df_risks_location.loc[y, 'Impact'])
            else:
                gbl[z].append(0)
                
for i in threatlist:
    df_nodes[i] = gbl[i]
    
df_nodes

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,Latitude,Longitude,Risks - Type,Risks - List,Circle_0,Box_1,Box_2
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,47.280,-122.20,0.12,0.000,0.00,0.00,0.00
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,45.660,-122.53,0.06,0.000,0.00,0.00,0.00
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,45.800,-118.53,0.06,0.000,0.00,0.00,0.00
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,47.160,-116.56,0.06,0.000,0.00,0.00,0.00
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,45.210,-107.77,0.06,0.000,0.00,0.00,0.00
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,47.810,-103.28,0.12,0.001,0.00,0.00,0.00
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,41.380,-109.79,0.06,0.000,0.00,0.00,0.00
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,42.680,-112.16,0.12,0.000,0.00,0.00,0.00
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,39.840,-112.25,0.06,0.000,0.00,0.00,0.00
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,40.710,-117.00,0.06,0.000,0.00,0.00,0.00


In [19]:
# Add column 'Total Risks' that sums all the risks from df_risks_type, df_risks_list, df_risks_location

df_nodes['Total Risks'] = df_nodes[['Risks - Type', 'Risks - List'] + threatlist].sum(axis=1)
df_nodes.loc[df_nodes['Total Risks'] > 1, 'Total Risks'] = 1
df_nodes['Node ID'] = df_nodes.index + 1 
df_nodes

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,Latitude,Longitude,Risks - Type,Risks - List,Circle_0,Box_1,Box_2,Total Risks
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,47.280,-122.20,0.12,0.000,0.00,0.00,0.00,0.120
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,45.660,-122.53,0.06,0.000,0.00,0.00,0.00,0.060
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,45.800,-118.53,0.06,0.000,0.00,0.00,0.00,0.060
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,47.160,-116.56,0.06,0.000,0.00,0.00,0.00,0.060
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,45.210,-107.77,0.06,0.000,0.00,0.00,0.00,0.060
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,47.810,-103.28,0.12,0.001,0.00,0.00,0.00,0.121
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,41.380,-109.79,0.06,0.000,0.00,0.00,0.00,0.060
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,42.680,-112.16,0.12,0.000,0.00,0.00,0.00,0.120
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,39.840,-112.25,0.06,0.000,0.00,0.00,0.00,0.060
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,40.710,-117.00,0.06,0.000,0.00,0.00,0.00,0.060


In [20]:
# df_nodes1 is nodes part 1 (black nodes) with supply and 0 demand

df_nodes1 = df_nodes.copy()
df_nodes1['Demand'] = 0
df_nodes1

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,Latitude,Longitude,Risks - Type,Risks - List,Circle_0,Box_1,Box_2,Total Risks
0,1,Washington AFB,End,Use,8500,0.00,0,0,0,No,47.280,-122.20,0.12,0.000,0.00,0.00,0.00,0.120
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,45.660,-122.53,0.06,0.000,0.00,0.00,0.00,0.060
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,45.800,-118.53,0.06,0.000,0.00,0.00,0.00,0.060
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,47.160,-116.56,0.06,0.000,0.00,0.00,0.00,0.060
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,45.210,-107.77,0.06,0.000,0.00,0.00,0.00,0.060
5,6,North Dakota AFB,End,Use,16000,0.00,0,0,0,No,47.810,-103.28,0.12,0.001,0.00,0.00,0.00,0.121
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,41.380,-109.79,0.06,0.000,0.00,0.00,0.00,0.060
7,8,Idaho AFB,End,Use,11500,0.00,0,0,0,No,42.680,-112.16,0.12,0.000,0.00,0.00,0.00,0.120
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,39.840,-112.25,0.06,0.000,0.00,0.00,0.00,0.060
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,40.710,-117.00,0.06,0.000,0.00,0.00,0.00,0.060


In [21]:
# df_nodes2 is nodes part 2 (blue nodes) with demand and 0 supply, current storage, storage capacity, total risks

df_nodes2 = df_nodes.copy()
df_nodes2['Node ID'] += nodes_len
df_nodes2['Node Name'] = df_nodes2['Node Name'] + ' Part 2'
df_nodes2['Supply'] = 0
df_nodes2['Storage Capacity'] = 0
df_nodes2['Current Storage'] = 0
df_nodes2['Total Risks'] = 0
df_nodes2

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,Latitude,Longitude,Risks - Type,Risks - List,Circle_0,Box_1,Box_2,Total Risks
0,50,Washington AFB Part 2,End,Use,8500,0,0,7650,0,No,47.280,-122.20,0.12,0.000,0.00,0.00,0.00,0
1,51,Oregon bulk storage 1 Part 2,Intermediate,Bulk Storage,46030,0,0,0,0,No,45.660,-122.53,0.06,0.000,0.00,0.00,0.00,0
2,52,Oregon bulk storage 2 Part 2,Intermediate,Bulk Storage,49907,0,0,0,0,No,45.800,-118.53,0.06,0.000,0.00,0.00,0.00,0
3,53,Idaho bulk storage Part 2,Intermediate,Bulk Storage,52720,0,0,0,0,No,47.160,-116.56,0.06,0.000,0.00,0.00,0.00,0
4,54,Montana bulk storage Part 2,Intermediate,Bulk Storage,54739,0,0,0,0,No,45.210,-107.77,0.06,0.000,0.00,0.00,0.00,0
5,55,North Dakota AFB Part 2,End,Use,16000,0,0,14400,0,No,47.810,-103.28,0.12,0.001,0.00,0.00,0.00,0
6,56,Wyoming bulk storage Part 2,Intermediate,Bulk Storage,53422,0,0,0,0,No,41.380,-109.79,0.06,0.000,0.00,0.00,0.00,0
7,57,Idaho AFB Part 2,End,Use,11500,0,0,10350,0,No,42.680,-112.16,0.12,0.000,0.00,0.00,0.00,0
8,58,Utah bulk storage Part 2,Intermediate,Bulk Storage,50100,0,0,0,0,No,39.840,-112.25,0.06,0.000,0.00,0.00,0.00,0
9,59,Nevada bulk storage 1 Part 2,Intermediate,Bulk Storage,48024,0,0,0,0,No,40.710,-117.00,0.06,0.000,0.00,0.00,0.00,0


In [22]:
# Reset df_nodes to be the concatenation of df_nodes1 and df_nodes2

df_nodes = pd.concat([df_nodes1, df_nodes2])
df_nodes = df_nodes.reset_index(drop = True)
df_nodes

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,Latitude,Longitude,Risks - Type,Risks - List,Circle_0,Box_1,Box_2,Total Risks
0,1,Washington AFB,End,Use,8500,0.00,0,0,0,No,47.28,-122.20,0.12,0.0,0.0,0.00,0.0,0.12
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,45.66,-122.53,0.06,0.0,0.0,0.00,0.0,0.06
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,45.80,-118.53,0.06,0.0,0.0,0.00,0.0,0.06
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,47.16,-116.56,0.06,0.0,0.0,0.00,0.0,0.06
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,45.21,-107.77,0.06,0.0,0.0,0.00,0.0,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,94,Maryland AFB Part 2,End,Use,16000,0.00,0,14400,0,No,38.29,-76.39,0.12,0.0,0.0,0.00,0.0,0.00
94,95,New York bulk storage Part 2,Intermediate,Bulk Storage,54165,0.00,0,0,0,No,43.04,-78.43,0.06,0.0,0.0,0.03,0.0,0.00
95,96,New York AFB Part 2,End,Use,15000,0.00,0,13500,0,No,40.61,-73.95,0.12,0.0,0.0,0.00,0.0,0.00
96,97,Massachusetts AFB Part 2,End,Use,18000,0.00,0,16200,0,No,42.20,-71.01,0.12,0.0,0.0,0.00,0.0,0.00


In [23]:
# Create df_nodes_risks to use later in the code

df_nodes_risks = pd.merge(df_nodes_orig, df_nodes, how = 'left', on = 'Node ID')
df_nodes_risks = (df_nodes_risks[['Node ID', 'Node Name_x', 'Description_x', 'Type_x', 'Throughput_x',
                 'Storage Capacity_x', 'Supply_x', 'Demand_x', 'Current Storage_x', 'Resupply_x', 
                 'Latitude_x', 'Longitude_x', 'Total Risks']])
df_nodes_risks.columns = [x.replace('_x', '') for x in df_nodes_risks.columns]
df_nodes_risks

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,Latitude,Longitude,Total Risks
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,47.280,-122.20,0.120
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,45.660,-122.53,0.060
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,45.800,-118.53,0.060
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,47.160,-116.56,0.060
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,45.210,-107.77,0.060
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,47.810,-103.28,0.121
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,41.380,-109.79,0.060
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,42.680,-112.16,0.120
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,39.840,-112.25,0.060
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,40.710,-117.00,0.060


In [24]:
# Create df_nodes_to_edges to use later in the code. This creates edges part 1 (blue edges).

df_nodes_to_edges = df_nodes_risks.copy()
df_nodes_to_edges = (df_nodes_to_edges.rename(columns = ({'Node ID': 'Start Node', 'Node Name': 'Edge Name', 
                                                         'Throughput': 'Capacity'})))
df_nodes_to_edges = df_nodes_to_edges[['Start Node', 'Edge Name', 'Capacity', 'Total Risks']]
df_nodes_to_edges['Edge Name'] = "NEW EDGE " + df_nodes_to_edges['Start Node'].astype(str)
df_nodes_to_edges['End Node'] = df_nodes_to_edges['Start Node'] + nodes_len
df_nodes_to_edges

,Start Node,Edge Name,Capacity,Total Risks,End Node
0,1,NEW EDGE 1,8500,0.120,50
1,2,NEW EDGE 2,46030,0.060,51
2,3,NEW EDGE 3,49907,0.060,52
3,4,NEW EDGE 4,52720,0.060,53
4,5,NEW EDGE 5,54739,0.060,54
5,6,NEW EDGE 6,16000,0.121,55
6,7,NEW EDGE 7,53422,0.060,56
7,8,NEW EDGE 8,11500,0.120,57
8,9,NEW EDGE 9,50100,0.060,58
9,10,NEW EDGE 10,48024,0.060,59


In [25]:
# Assign the 'risks - type' value to each edge

df_comb1 = pd.merge(df_edges, df_risks_type, how = 'left', on = 'Type')
df_comb1['Expected_Impact'] = df_comb1['Probability'] * df_comb1['Impact']
df_comb1 = df_comb1[['Edge ID', 'Expected_Impact']]
df_comb1 = df_comb1.groupby(['Edge ID']).sum()
df_comb1

,Expected_Impact
Edge ID,
1,0.22
2,0.14
3,0.45
4,0.14
5,0.22
...,...
119,0.22
120,0.45
121,0.14


In [26]:
# Add the 'risks - type' values to df_edges

df_edges = pd.merge(df_edges, df_comb1, how = 'left', on = 'Edge ID')
df_edges = df_edges.rename(columns = {'Expected_Impact': 'Risks - Type'})
df_edges

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Risks - Type
0,1,2,10,Pipeline 1,blue,Pipeline,65627,0.22
1,2,2,11,Railroad 1,green,Railroad,64163,0.14
2,3,3,6,Truck 1,red,Truck,37775,0.45
3,4,4,2,Railroad 2,green,Railroad,53411,0.14
4,5,5,8,Pipeline 2,blue,Pipeline,25437,0.22
...,...,...,...,...,...,...,...,...
118,119,46,48,Pipeline 45,blue,Pipeline,72395,0.22
119,120,46,48,Truck 25,red,Truck,46521,0.45
120,121,47,48,Railroad 44,green,Railroad,20357,0.14
121,122,47,48,Pipeline 46,blue,Pipeline,65910,0.22


In [27]:
# Add column 'Edge & ID' that combines the word 'Edge' with the edge ID in order to merge with df_risks_list

df_edges['Edge & ID'] = 'Edge ' + df_edges['Edge ID'].astype(str)
df_edges

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Risks - Type,Edge & ID
0,1,2,10,Pipeline 1,blue,Pipeline,65627,0.22,Edge 1
1,2,2,11,Railroad 1,green,Railroad,64163,0.14,Edge 2
2,3,3,6,Truck 1,red,Truck,37775,0.45,Edge 3
3,4,4,2,Railroad 2,green,Railroad,53411,0.14,Edge 4
4,5,5,8,Pipeline 2,blue,Pipeline,25437,0.22,Edge 5
...,...,...,...,...,...,...,...,...,...
118,119,46,48,Pipeline 45,blue,Pipeline,72395,0.22,Edge 119
119,120,46,48,Truck 25,red,Truck,46521,0.45,Edge 120
120,121,47,48,Railroad 44,green,Railroad,20357,0.14,Edge 121
121,122,47,48,Pipeline 46,blue,Pipeline,65910,0.22,Edge 122


In [28]:
# Assign the 'risks - list' value to each edge

df_comb1 = pd.merge(df_edges, df_risks_list, how = 'left', left_on = 'Edge & ID', right_on = 'Node/Edge & ID')
df_comb1 = df_comb1.fillna(0)
df_comb1['Expected_Impact'] = (df_comb1['Probability'] * df_comb1['Impact'])
df_comb1 = df_comb1[['Edge ID', 'Expected_Impact']]
df_comb1 = df_comb1.groupby(['Edge ID']).sum()
df_comb1

,Expected_Impact
Edge ID,
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
...,...
119,0.0
120,0.0
121,0.0


In [29]:
# Add the 'risks - list' values to df_edges

df_edges = pd.merge(df_edges, df_comb1, how = 'left', on = 'Edge ID')
df_edges = df_edges.rename(columns = {'Expected_Impact': 'Risks - List'})
df_edges.pop('Edge & ID')
df_edges

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Risks - Type,Risks - List
0,1,2,10,Pipeline 1,blue,Pipeline,65627,0.22,0.0
1,2,2,11,Railroad 1,green,Railroad,64163,0.14,0.0
2,3,3,6,Truck 1,red,Truck,37775,0.45,0.0
3,4,4,2,Railroad 2,green,Railroad,53411,0.14,0.0
4,5,5,8,Pipeline 2,blue,Pipeline,25437,0.22,0.0
...,...,...,...,...,...,...,...,...,...
118,119,46,48,Pipeline 45,blue,Pipeline,72395,0.22,0.0
119,120,46,48,Truck 25,red,Truck,46521,0.45,0.0
120,121,47,48,Railroad 44,green,Railroad,20357,0.14,0.0
121,122,47,48,Pipeline 46,blue,Pipeline,65910,0.22,0.0


In [30]:
# Combine df_nodes and df_edges to eventually apply nodes' risks - locations to edges

df_comb1 = pd.merge(df_nodes, df_edges, how = 'right', left_on = 'Node ID', right_on = 'Start Node')
df_comb1

,Node ID,Node Name,Description_x,Type_x,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Total Risks,Edge ID,Start Node,End Node,Edge Name,Description_y,Type_y,Capacity,Risks - Type_y,Risks - List_y
0,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,0.06,1,2,10,Pipeline 1,blue,Pipeline,65627,0.22,0.0
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,0.06,2,2,11,Railroad 1,green,Railroad,64163,0.14,0.0
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,...,0.06,3,3,6,Truck 1,red,Truck,37775,0.45,0.0
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,...,0.06,4,4,2,Railroad 2,green,Railroad,53411,0.14,0.0
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,...,0.06,5,5,8,Pipeline 2,blue,Pipeline,25437,0.22,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,46,New York bulk storage,Intermediate,Bulk Storage,54165,2708.25,0,0,1148,No,...,0.09,119,46,48,Pipeline 45,blue,Pipeline,72395,0.22,0.0
119,46,New York bulk storage,Intermediate,Bulk Storage,54165,2708.25,0,0,1148,No,...,0.09,120,46,48,Truck 25,red,Truck,46521,0.45,0.0
120,47,New York AFB,End,Use,15000,0.00,0,0,0,No,...,0.12,121,47,48,Railroad 44,green,Railroad,20357,0.14,0.0
121,47,New York AFB,End,Use,15000,0.00,0,0,0,No,...,0.12,122,47,48,Pipeline 46,blue,Pipeline,65910,0.22,0.0


In [31]:
# Combine df_nodes and df_edges to eventually apply nodes' risks - locations to edges

df_comb2 = pd.merge(df_nodes, df_edges, how = 'right', left_on = 'Node ID', right_on = 'End Node')
df_comb2

,Node ID,Node Name,Description_x,Type_x,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Total Risks,Edge ID,Start Node,End Node,Edge Name,Description_y,Type_y,Capacity,Risks - Type_y,Risks - List_y
0,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,...,0.060,1,2,10,Pipeline 1,blue,Pipeline,65627,0.22,0.0
1,11,California bulk storage,Intermediate,Bulk Storage,53527,2676.35,0,0,967,No,...,0.060,2,2,11,Railroad 1,green,Railroad,64163,0.14,0.0
2,6,North Dakota AFB,End,Use,16000,0.00,0,0,0,No,...,0.121,3,3,6,Truck 1,red,Truck,37775,0.45,0.0
3,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,0.060,4,4,2,Railroad 2,green,Railroad,53411,0.14,0.0
4,8,Idaho AFB,End,Use,11500,0.00,0,0,0,No,...,0.120,5,5,8,Pipeline 2,blue,Pipeline,25437,0.22,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,48,Massachusetts AFB,End,Use,18000,0.00,0,0,0,No,...,0.120,119,46,48,Pipeline 45,blue,Pipeline,72395,0.22,0.0
119,48,Massachusetts AFB,End,Use,18000,0.00,0,0,0,No,...,0.120,120,46,48,Truck 25,red,Truck,46521,0.45,0.0
120,48,Massachusetts AFB,End,Use,18000,0.00,0,0,0,No,...,0.120,121,47,48,Railroad 44,green,Railroad,20357,0.14,0.0
121,48,Massachusetts AFB,End,Use,18000,0.00,0,0,0,No,...,0.120,122,47,48,Pipeline 46,blue,Pipeline,65910,0.22,0.0


In [32]:
# Combine comb1 and comb2 then filter to see total risks - location for each edge

df_comb3 = pd.concat([df_comb1, df_comb2])
df_comb3['Edge_Risks_Location'] = (df_comb3['Total Risks'] - df_comb3['Risks - Type_x'] - df_comb3['Risks - List_x'])
df_comb3 = df_comb3[['Edge ID', 'Edge_Risks_Location']]
df_comb3 = df_comb3.groupby(['Edge ID']).sum()
df_comb3

,Edge_Risks_Location
Edge ID,
1,0.000000e+00
2,0.000000e+00
3,8.673617e-19
4,0.000000e+00
5,0.000000e+00
...,...
119,3.000000e-02
120,3.000000e-02
121,0.000000e+00


In [33]:
# Combine df_edges and df_comb3 and create 'Total Risks' column

df_edges = pd.merge(df_edges, df_comb3, how = 'inner', on = 'Edge ID')
df_edges = df_edges.rename(columns = {'Edge_Risks_Location': 'Risks - Location'})
df_edges['Total Risks'] = df_edges[['Risks - Type', 'Risks - List', 'Risks - Location']].sum(axis=1)
df_edges.loc[df_edges['Total Risks'] > 1, 'Total Risks'] = 1
df_edges

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Risks - Type,Risks - List,Risks - Location,Total Risks
0,1,2,10,Pipeline 1,blue,Pipeline,65627,0.22,0.0,0.000000e+00,0.22
1,2,2,11,Railroad 1,green,Railroad,64163,0.14,0.0,0.000000e+00,0.14
2,3,3,6,Truck 1,red,Truck,37775,0.45,0.0,8.673617e-19,0.45
3,4,4,2,Railroad 2,green,Railroad,53411,0.14,0.0,0.000000e+00,0.14
4,5,5,8,Pipeline 2,blue,Pipeline,25437,0.22,0.0,0.000000e+00,0.22
...,...,...,...,...,...,...,...,...,...,...,...
118,119,46,48,Pipeline 45,blue,Pipeline,72395,0.22,0.0,3.000000e-02,0.25
119,120,46,48,Truck 25,red,Truck,46521,0.45,0.0,3.000000e-02,0.48
120,121,47,48,Railroad 44,green,Railroad,20357,0.14,0.0,0.000000e+00,0.14
121,122,47,48,Pipeline 46,blue,Pipeline,65910,0.22,0.0,0.000000e+00,0.22


In [34]:
# Set 'Start Node' to new nodes (part 2 blue nodes) to create edges part 2 (black edges)

df_edges['Start Node'] += nodes_len
df_edges

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Risks - Type,Risks - List,Risks - Location,Total Risks
0,1,51,10,Pipeline 1,blue,Pipeline,65627,0.22,0.0,0.000000e+00,0.22
1,2,51,11,Railroad 1,green,Railroad,64163,0.14,0.0,0.000000e+00,0.14
2,3,52,6,Truck 1,red,Truck,37775,0.45,0.0,8.673617e-19,0.45
3,4,53,2,Railroad 2,green,Railroad,53411,0.14,0.0,0.000000e+00,0.14
4,5,54,8,Pipeline 2,blue,Pipeline,25437,0.22,0.0,0.000000e+00,0.22
...,...,...,...,...,...,...,...,...,...,...,...
118,119,95,48,Pipeline 45,blue,Pipeline,72395,0.22,0.0,3.000000e-02,0.25
119,120,95,48,Truck 25,red,Truck,46521,0.45,0.0,3.000000e-02,0.48
120,121,96,48,Railroad 44,green,Railroad,20357,0.14,0.0,0.000000e+00,0.14
121,122,96,48,Pipeline 46,blue,Pipeline,65910,0.22,0.0,0.000000e+00,0.22


In [35]:
# Combine df_edges and df_nodes_to_edges to get all black (part 1) and blue (part 2) edges together in df_edges

df_edges = pd.concat([df_edges, df_nodes_to_edges])
df_edges = df_edges.reset_index(drop = True)
df_edges['Edge ID'] = df_edges.index + 1
df_edges

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Risks - Type,Risks - List,Risks - Location,Total Risks
0,1,51,10,Pipeline 1,blue,Pipeline,65627,0.22,0.0,0.000000e+00,0.22
1,2,51,11,Railroad 1,green,Railroad,64163,0.14,0.0,0.000000e+00,0.14
2,3,52,6,Truck 1,red,Truck,37775,0.45,0.0,8.673617e-19,0.45
3,4,53,2,Railroad 2,green,Railroad,53411,0.14,0.0,0.000000e+00,0.14
4,5,54,8,Pipeline 2,blue,Pipeline,25437,0.22,0.0,0.000000e+00,0.22
...,...,...,...,...,...,...,...,...,...,...,...
167,168,45,94,NEW EDGE 45,NaN,NaN,16000,NaN,NaN,NaN,0.12
168,169,46,95,NEW EDGE 46,NaN,NaN,54165,NaN,NaN,NaN,0.09
169,170,47,96,NEW EDGE 47,NaN,NaN,15000,NaN,NaN,NaN,0.12
170,171,48,97,NEW EDGE 48,NaN,NaN,18000,NaN,NaN,NaN,0.12


In [36]:
# Create df_edges_risks to use later in the code

df_edges_risks = df_edges.copy()
df_edges_risks

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Risks - Type,Risks - List,Risks - Location,Total Risks
0,1,51,10,Pipeline 1,blue,Pipeline,65627,0.22,0.0,0.000000e+00,0.22
1,2,51,11,Railroad 1,green,Railroad,64163,0.14,0.0,0.000000e+00,0.14
2,3,52,6,Truck 1,red,Truck,37775,0.45,0.0,8.673617e-19,0.45
3,4,53,2,Railroad 2,green,Railroad,53411,0.14,0.0,0.000000e+00,0.14
4,5,54,8,Pipeline 2,blue,Pipeline,25437,0.22,0.0,0.000000e+00,0.22
...,...,...,...,...,...,...,...,...,...,...,...
167,168,45,94,NEW EDGE 45,NaN,NaN,16000,NaN,NaN,NaN,0.12
168,169,46,95,NEW EDGE 46,NaN,NaN,54165,NaN,NaN,NaN,0.09
169,170,47,96,NEW EDGE 47,NaN,NaN,15000,NaN,NaN,NaN,0.12
170,171,48,97,NEW EDGE 48,NaN,NaN,18000,NaN,NaN,NaN,0.12


In [37]:
# Merge df_edges and df_nodes (twice) to become the new df_edges

df_edges = pd.merge(df_edges, df_nodes, how = 'left', left_on = 'Start Node', right_on = 'Node ID')
df_edges = pd.merge(df_edges, df_nodes, how = 'left', left_on = 'End Node', right_on = 'Node ID')
df_edges = (df_edges[['Edge ID', 'Start Node', 'End Node', 'Edge Name', 'Description_x', 'Type_x', 'Capacity', 
                      'Risks - Type_x', 'Risks - List_x', 'Risks - Location', 'Total Risks_x', 'Type_y', 'Type']])
df_edges = df_edges.rename(columns = {'Type_y': 'Start Node Type', 'Type': 'End Node Type'})
df_edges.columns = [x.replace('_x', '') for x in df_edges.columns]
df_edges

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Risks - Type,Risks - List,Risks - Location,Total Risks,Start Node Type,End Node Type
0,1,51,10,Pipeline 1,blue,Pipeline,65627,0.22,0.0,0.000000e+00,0.22,Bulk Storage,Bulk Storage
1,2,51,11,Railroad 1,green,Railroad,64163,0.14,0.0,0.000000e+00,0.14,Bulk Storage,Bulk Storage
2,3,52,6,Truck 1,red,Truck,37775,0.45,0.0,8.673617e-19,0.45,Bulk Storage,Use
3,4,53,2,Railroad 2,green,Railroad,53411,0.14,0.0,0.000000e+00,0.14,Bulk Storage,Bulk Storage
4,5,54,8,Pipeline 2,blue,Pipeline,25437,0.22,0.0,0.000000e+00,0.22,Bulk Storage,Use
...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,168,45,94,NEW EDGE 45,NaN,NaN,16000,NaN,NaN,NaN,0.12,Use,Use
168,169,46,95,NEW EDGE 46,NaN,NaN,54165,NaN,NaN,NaN,0.09,Bulk Storage,Bulk Storage
169,170,47,96,NEW EDGE 47,NaN,NaN,15000,NaN,NaN,NaN,0.12,Use,Use
170,171,48,97,NEW EDGE 48,NaN,NaN,18000,NaN,NaN,NaN,0.12,Use,Use


In [38]:
# Create df_edges_copy to use later in the code

df_edges_copy = df_edges.copy()
df_edges_copy = df_edges_copy[['Start Node', 'End Node', 'Capacity', 'Total Risks']]
df_edges_copy = (df_edges_copy.rename(columns = {'Start Node': 'Start Node ID', 'End Node': 'End Node ID', 
                        'Capacity': 'End Node Capacity', 'Total Risks': 'Edge Risk'}))
df_edges_copy['End Node Expected Capacity'] = (df_edges_copy['End Node Capacity'] *
                        (1 - df_edges_copy['Edge Risk']))
df_edges_copy

,Start Node ID,End Node ID,End Node Capacity,Edge Risk,End Node Expected Capacity
0,51,10,65627,0.22,51189.06
1,51,11,64163,0.14,55180.18
2,52,6,37775,0.45,20776.25
3,53,2,53411,0.14,45933.46
4,54,8,25437,0.22,19840.86
...,...,...,...,...,...
167,45,94,16000,0.12,14080.00
168,46,95,54165,0.09,49290.15
169,47,96,15000,0.12,13200.00
170,48,97,18000,0.12,15840.00


In [39]:
# Create df_nodes_copy to use later in the code

df_nodes_copy = df_nodes.copy()
df_nodes_copy['Supply/Demand/Storage'] = (df_nodes_copy['Current Storage'] + df_nodes_copy['Supply'] 
                                        - df_nodes_copy['Demand'])
df_nodes_copy = df_nodes_copy.rename(columns = {'Node ID': 'Supply/Demand/Storage Node ID'})
df_nodes_copy = df_nodes_copy[['Supply/Demand/Storage Node ID', 'Supply/Demand/Storage']]
df_nodes_copy

,Supply/Demand/Storage Node ID,Supply/Demand/Storage
0,1,0
1,2,1521
2,3,614
3,4,955
4,5,735
...,...,...
93,94,-14400
94,95,0
95,96,-13500
96,97,-16200


In [40]:
# Create empty lists to use in min cost flow calculation

start_nodes = []
end_nodes = []
capacity = []
expected_capacity = []
supplies = []
id_supplies = []
risk = []
remaining_capacity = []
remaining_expected_capacity = []

In [41]:
# Append lists in the correct format

for i in range(len(df_edges_copy)):
    start_nodes.append(int(df_edges_copy.loc[i, 'Start Node ID']))
    end_nodes.append(int(df_edges_copy.loc[i, 'End Node ID']))
    capacity.append(int(float(df_edges_copy.loc[i, 'End Node Capacity'])))
    expected_capacity.append(int(float(df_edges_copy.loc[i, 'End Node Expected Capacity'])))
    risk.append(int(1000 * float(df_edges_copy.loc[i, 'Edge Risk'])))

In [42]:
# Append lists in the correct format

for i in range(len(df_nodes_copy)):
    supplies.append(int(df_nodes_copy.loc[i, 'Supply/Demand/Storage']))
    id_supplies.append(int(df_nodes_copy.loc[i, 'Supply/Demand/Storage Node ID']))

In [43]:
# Baseline - part 1: Max flow with min cost with all node/edge risks equal to 0

flow = pywrapgraph.SimpleMinCostFlow()

# Add each arc
for i in range(len(start_nodes)):
    flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i], capacity[i], 0)

# Add node supplies
for i in range(len(supplies)):
    flow.SetNodeSupply(int(id_supplies[i]), supplies[i])

start_node_output = []
end_node_output = []
flow_output = []
capacity_output = []
risk_output = []

start_node_output_v2 = []
end_node_output_v2 = []
flow_output_v2 = []
capacity_output_v2 = []
risk_output_v2 = []

# Find the minimum cost flow
if flow.SolveMaxFlowWithMinCost():
    for i in range(flow.NumArcs()):
        remaining_capacity.append(capacity[i] - flow.Flow(i))
        if start_nodes[i] > nodes_len:
            start_node_output.append(flow.Tail(i))
            end_node_output.append(flow.Head(i))
            flow_output.append(flow.Flow(i))
            capacity_output.append(flow.Capacity(i))
            risk_calc = (flow.UnitCost(i)/1000)
            risk_output.append(risk_calc)
        else:
            start_node_output_v2.append(flow.Tail(i))
            end_node_output_v2.append(flow.Head(i))
            flow_output_v2.append(flow.Flow(i))
            capacity_output_v2.append(flow.Capacity(i))
            risk_calc = (flow.UnitCost(i)/1000)
            risk_output_v2.append(risk_calc)

    total_flow_output_baseline_part1 = sum(flow_output) + sum(flow_output_v2)
    accum_risk_baseline_part1 = float(flow.OptimalCost()) / 1000
                
# Create df_baseline_part1 that includes flow along "true" edges (black edges part 2)
df_baseline_part1 = (pd.DataFrame(zip(start_node_output, end_node_output, flow_output, capacity_output, risk_output), 
                             columns = ['Start Node', 'End Node', 'Flow', 'Capacity', 'Risk']))
df_baseline_part1['Start Node'] = df_baseline_part1['Start Node'] - nodes_len
df_baseline_part1['Edge ID'] = df_baseline_part1.index + 1
df_baseline_part1

,Start Node,End Node,Flow,Capacity,Risk,Edge ID
0,2,10,3585,65627,0.0,1
1,2,11,1521,64163,0.0,2
2,3,6,14303,37775,0.0,3
3,4,2,3585,53411,0.0,4
4,5,8,10350,25437,0.0,5
...,...,...,...,...,...,...
118,46,48,0,72395,0.0,119
119,46,48,0,46521,0.0,120
120,47,48,1500,20357,0.0,121
121,47,48,0,65910,0.0,122


In [44]:
# Create df_baseline_part1_v2 that includes flow along "fake" edges (blue edges part 1)

df_baseline_part1_v2 = (pd.DataFrame(zip(start_node_output_v2, end_node_output_v2, flow_output_v2, capacity_output_v2, 
                   risk_output_v2), columns = ['Start Node', 'End Node', 'Flow', 'Capacity', 'Risk']))
df_baseline_part1_v2

,Start Node,End Node,Flow,Capacity,Risk
0,1,50,0,8500,0.0
1,2,51,5106,46030,0.0
2,3,52,14303,49907,0.0
3,4,53,3585,52720,0.0
4,5,54,10350,54739,0.0
5,6,55,14400,16000,0.0
6,7,56,0,53422,0.0
7,8,57,10350,11500,0.0
8,9,58,6230,50100,0.0
9,10,59,3978,48024,0.0


In [45]:
# Add baseline flow part 1 to df_nodes_final

df_nodes_final = (pd.merge(df_nodes_risks, df_baseline_part1_v2, how = 'left', left_on = 'Node ID', 
                           right_on = 'Start Node'))
df_nodes_final = df_nodes_final.drop(['Start Node', 'End Node', 'Capacity', 'Risk'], axis = 1)
df_nodes_final = df_nodes_final.rename(columns = {'Flow': 'Flow - Baseline part 1'})
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,Latitude,Longitude,Total Risks,Flow - Baseline part 1
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,47.280,-122.20,0.120,0
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,45.660,-122.53,0.060,5106
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,45.800,-118.53,0.060,14303
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,47.160,-116.56,0.060,3585
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,45.210,-107.77,0.060,10350
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,47.810,-103.28,0.121,14400
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,41.380,-109.79,0.060,0
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,42.680,-112.16,0.120,10350
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,39.840,-112.25,0.060,6230
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,40.710,-117.00,0.060,3978


In [46]:
# Create df_baseline_part1_gephi from df_baseline_part1 to format for csv export and gephi import

df_baseline_part1_gephi = df_baseline_part1[df_baseline_part1['Flow'] > 0]
df_baseline_part1_gephi = df_baseline_part1_gephi[['Start Node', 'End Node', 'Flow']]
df_baseline_part1_gephi['Start-End Node'] = (df_baseline_part1_gephi['Start Node'].astype(str) + ' - ' + 
                                             df_baseline_part1_gephi['End Node'].astype(str))
df_baseline_part1_gephi

,Start Node,End Node,Flow,Start-End Node
0,2,10,3585,2 - 10
1,2,11,1521,2 - 11
2,3,6,14303,3 - 6
3,4,2,3585,4 - 2
4,5,8,10350,5 - 8
8,9,4,2630,9 - 4
10,9,13,3600,9 - 13
13,10,12,3978,10 - 12
14,11,12,2488,11 - 12
18,14,3,14303,14 - 3


In [47]:
# Find the flow from each node

df_baseline_part1_start_node_flow = df_baseline_part1[['Start Node', 'Flow']]
df_baseline_part1_start_node_flow = df_baseline_part1_start_node_flow.groupby(['Start Node']).sum()
df_baseline_part1_start_node_flow.reset_index(inplace = True)
df_baseline_part1_start_node_flow

,Start Node,Flow
0,2,5106
1,3,14303
2,4,3585
3,5,10350
4,7,0
5,9,6230
6,10,3978
7,11,2488
8,12,0
9,14,14303


In [48]:
# Find the flow to each node

df_baseline_part1_end_node_flow = df_baseline_part1[['End Node', 'Flow']]
df_baseline_part1_end_node_flow = df_baseline_part1_end_node_flow.groupby(['End Node']).sum()
df_baseline_part1_end_node_flow.reset_index(inplace = True)
df_baseline_part1_end_node_flow

,End Node,Flow
0,1,0
1,2,3585
2,3,14303
3,4,2630
4,5,9615
5,6,14400
6,7,0
7,8,10350
8,9,4286
9,10,3585


In [49]:
# Combine flow to/from each node

df_baseline_part1_net_flow = (pd.merge(df_baseline_part1_start_node_flow, df_baseline_part1_end_node_flow, 
                                how = 'outer', left_on = 'Start Node', right_on = 'End Node'))
df_baseline_part1_net_flow = df_baseline_part1_net_flow.fillna(0)
df_baseline_part1_net_flow['Node ID'] = df_baseline_part1_net_flow[['Start Node', 'End Node']].max(axis = 1)
df_baseline_part1_net_flow

,Start Node,Flow_x,End Node,Flow_y,Node ID
0,2.0,5106.0,2.0,3585.0,2.0
1,3.0,14303.0,3.0,14303.0,3.0
2,4.0,3585.0,4.0,2630.0,4.0
3,5.0,10350.0,5.0,9615.0,5.0
4,7.0,0.0,7.0,0.0,7.0
5,9.0,6230.0,9.0,4286.0,9.0
6,10.0,3978.0,10.0,3585.0,10.0
7,11.0,2488.0,11.0,1521.0,11.0
8,12.0,0.0,12.0,13405.0,12.0
9,14.0,14303.0,14.0,13776.0,14.0


In [50]:
# Calculate net flow from each node

df_baseline_part1_net_flow['Net Flow - Baseline part 1'] = (df_baseline_part1_net_flow['Flow_x'] - 
                                                            df_baseline_part1_net_flow['Flow_y'])
df_baseline_part1_net_flow = df_baseline_part1_net_flow[['Node ID', 'Net Flow - Baseline part 1']]
df_baseline_part1_net_flow

,Node ID,Net Flow - Baseline part 1
0,2.0,1521.0
1,3.0,0.0
2,4.0,955.0
3,5.0,735.0
4,7.0,0.0
5,9.0,1944.0
6,10.0,393.0
7,11.0,967.0
8,12.0,-13405.0
9,14.0,527.0


In [51]:
# Add columns in df_nodes_final and update values

df_nodes_final = pd.merge(df_nodes_final, df_baseline_part1_net_flow, how = 'outer', on = 'Node ID')
df_nodes_final['Remaining Storage - Baseline part 1'] = (df_nodes_final['Current Storage'] - 
                                                  df_nodes_final['Net Flow - Baseline part 1'])
df_nodes_final.loc[df_nodes_final['Remaining Storage - Baseline part 1'] < 0, 'Remaining Storage - Baseline part 1'] = 0
df_nodes_final.loc[df_nodes_final['Demand'] > 0, 'Remaining Storage - Baseline part 1'] = 0
df_nodes_final.loc[df_nodes_final['Demand'] > 0, 'Remaining Storage Capacity - Baseline part 1'] = 0
df_nodes_final['Remaining Storage Capacity - Baseline part 1'] = (df_nodes_final['Storage Capacity'] - 
                                                           df_nodes_final['Remaining Storage - Baseline part 1'])
df_nodes_final['New Supplies - Baseline'] = (df_nodes_final['Supply'] - 
                                             df_nodes_final['Net Flow - Baseline part 1'])
df_nodes_final.loc[df_nodes_final['New Supplies - Baseline'] < 0, 'New Supplies - Baseline'] = 0
df_nodes_final.loc[df_nodes_final['Resupply'] == 'No', 'New Supplies - Baseline'] = 0
df_nodes_final['New Demand - Baseline'] = df_nodes_final['Remaining Storage Capacity - Baseline part 1']
df_nodes_final['New Supply/Demand - Baseline'] = (df_nodes_final['New Supplies - Baseline'] - 
                                                  df_nodes_final['New Demand - Baseline'])
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,Latitude,Longitude,Total Risks,Flow - Baseline part 1,Net Flow - Baseline part 1,Remaining Storage - Baseline part 1,Remaining Storage Capacity - Baseline part 1,New Supplies - Baseline,New Demand - Baseline,New Supply/Demand - Baseline
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,47.280,-122.20,0.120,0,0.0,0.0,0.00,0.0,0.00,0.00
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,45.660,-122.53,0.060,5106,1521.0,0.0,2301.50,0.0,2301.50,-2301.50
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,45.800,-118.53,0.060,14303,0.0,614.0,1881.35,0.0,1881.35,-1881.35
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,47.160,-116.56,0.060,3585,955.0,0.0,2636.00,0.0,2636.00,-2636.00
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,45.210,-107.77,0.060,10350,735.0,0.0,2736.95,0.0,2736.95,-2736.95
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,47.810,-103.28,0.121,14400,-14400.0,0.0,0.00,0.0,0.00,0.00
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,41.380,-109.79,0.060,0,0.0,798.0,1873.10,0.0,1873.10,-1873.10
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,42.680,-112.16,0.120,10350,-10350.0,0.0,0.00,0.0,0.00,0.00
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,39.840,-112.25,0.060,6230,1944.0,0.0,2505.00,0.0,2505.00,-2505.00
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,40.710,-117.00,0.060,3978,393.0,0.0,2401.20,0.0,2401.20,-2401.20


In [52]:
# Create df_new_supplies to capture supply from all new nodes (blue nodes part 2)

df_new_supplies = df_nodes_final.copy()
df_new_supplies = df_new_supplies[['Node ID', 'New Supply/Demand - Baseline']]
df_new_supplies['Node ID'] += nodes_len
df_new_supplies['New Supply/Demand - Baseline'] = 0
df_new_supplies

,Node ID,New Supply/Demand - Baseline
0,50,0
1,51,0
2,52,0
3,53,0
4,54,0
5,55,0
6,56,0
7,57,0
8,58,0
9,59,0


In [53]:
# Create df_new_supplies_all to find supply from all black (part 1) and blue (part 2) nodes

df_new_supplies_all = pd.concat([df_nodes_final, df_new_supplies])
df_new_supplies_all = df_new_supplies_all.reset_index(drop = True)
df_new_supplies_all

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,Latitude,Longitude,Total Risks,Flow - Baseline part 1,Net Flow - Baseline part 1,Remaining Storage - Baseline part 1,Remaining Storage Capacity - Baseline part 1,New Supplies - Baseline,New Demand - Baseline,New Supply/Demand - Baseline
0,1,Washington AFB,End,Use,8500.0,0.00,0.0,7650.0,0.0,No,47.28,-122.20,0.12,0.0,0.0,0.0,0.00,0.0,0.00,0.00
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030.0,2301.50,0.0,0.0,1521.0,No,45.66,-122.53,0.06,5106.0,1521.0,0.0,2301.50,0.0,2301.50,-2301.50
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907.0,2495.35,0.0,0.0,614.0,No,45.80,-118.53,0.06,14303.0,0.0,614.0,1881.35,0.0,1881.35,-1881.35
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720.0,2636.00,0.0,0.0,955.0,No,47.16,-116.56,0.06,3585.0,955.0,0.0,2636.00,0.0,2636.00,-2636.00
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739.0,2736.95,0.0,0.0,735.0,No,45.21,-107.77,0.06,10350.0,735.0,0.0,2736.95,0.0,2736.95,-2736.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
94,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
95,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
96,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00


In [54]:
# Create empty lists to use in the second min cost flow calculation

new_supplies = []
new_id_supplies = []

In [55]:
# Append lists in the correct format

for i in range(len(df_new_supplies_all)):
    new_supplies.append(int(df_new_supplies_all.loc[i, 'New Supply/Demand - Baseline']))
    new_id_supplies.append(int(df_new_supplies_all.loc[i, 'Node ID']))

In [56]:
# Create df_new_inputs in order to create lists for second min cost flow calculation

list = ({'Start Nodes': start_nodes, 'End Nodes': end_nodes, 'Remaining Capacity': remaining_capacity, 
               'Risk': risk})
df_new_inputs = pd.DataFrame(list)
df_new_inputs

,Start Nodes,End Nodes,Remaining Capacity,Risk
0,51,10,62042,219
1,51,11,62642,140
2,52,6,23472,450
3,53,2,49826,140
4,54,8,15087,219
...,...,...,...,...
167,45,94,1600,120
168,46,95,50963,90
169,47,96,0,120
170,48,97,1800,120


In [57]:
# Create empty lists to use in the second min cost flow calculation

start_nodes_2 = []
end_nodes_2 = []
remaining_capacity_2 = []
risk_2 = []

In [58]:
# Append lists in the correct format

for i in range(len(df_new_inputs)):
    start_nodes_2.append(int(df_new_inputs.loc[i, 'Start Nodes']))
    end_nodes_2.append(int(df_new_inputs.loc[i, 'End Nodes']))
    remaining_capacity_2.append(int(float(df_new_inputs.loc[i, 'Remaining Capacity'])))
    risk_2.append(int(float(df_new_inputs.loc[i, 'Risk'])))

In [59]:
# Baseline - part 2: Max flow with min cost with all node/edge risks equal to 0

flow = pywrapgraph.SimpleMinCostFlow()

# Add each arc
for i in range(len(start_nodes)):
    flow.AddArcWithCapacityAndUnitCost(start_nodes_2[i], end_nodes_2[i], remaining_capacity_2[i], 0)

# Add node supplies
for i in range(len(new_supplies)):
    flow.SetNodeSupply(int(new_id_supplies[i]), new_supplies[i])

start_node_output = []
end_node_output = []
flow_output = []
capacity_output = []
risk_output = []

start_node_output_v2 = []
end_node_output_v2 = []
flow_output_v2 = []
capacity_output_v2 = []
risk_output_v2 = []

# Find the minimum cost flow
if flow.SolveMaxFlowWithMinCost():
    for i in range(flow.NumArcs()):
        if start_nodes[i] > nodes_len:
            start_node_output.append(flow.Tail(i))
            end_node_output.append(flow.Head(i))
            flow_output.append(flow.Flow(i))
            capacity_output.append(flow.Capacity(i))
            risk_calc = (flow.UnitCost(i)/1000)
            risk_output.append(risk_calc)
        else:
            start_node_output_v2.append(flow.Tail(i))
            end_node_output_v2.append(flow.Head(i))
            flow_output_v2.append(flow.Flow(i))
            capacity_output_v2.append(flow.Capacity(i))
            risk_calc = (flow.UnitCost(i)/1000)
            risk_output_v2.append(risk_calc)

    total_flow_output_baseline_part2 = sum(flow_output) + sum(flow_output_v2)
    total_flow_output_baseline = total_flow_output_baseline_part1 + total_flow_output_baseline_part2
    accum_risk_baseline_part2 = float(flow.OptimalCost()) / 1000
    accum_risk_baseline = accum_risk_baseline_part1 + accum_risk_baseline_part2
    accum_risk_percent_baseline = accum_risk_baseline / total_flow_output_baseline
                
# Create df_baseline_part2 that includes flow along "true" nodes (black edges part 2)
df_baseline_part2 = (pd.DataFrame(zip(start_node_output, end_node_output, flow_output, capacity_output, risk_output), 
                             columns = ['Start Node', 'End Node', 'Flow', 'Capacity', 'Risk']))
df_baseline_part2['Start Node'] = df_baseline_part2['Start Node'] - nodes_len
df_baseline_part2['Edge ID'] = df_baseline_part2.index + 1
df_baseline_part2

,Start Node,End Node,Flow,Capacity,Risk,Edge ID
0,2,10,0,62042,0.0,1
1,2,11,0,62642,0.0,2
2,3,6,0,23472,0.0,3
3,4,2,0,49826,0.0,4
4,5,8,0,15087,0.0,5
...,...,...,...,...,...,...
118,46,48,0,72395,0.0,119
119,46,48,0,46521,0.0,120
120,47,48,0,18857,0.0,121
121,47,48,0,65910,0.0,122


In [60]:
# Create df_baseline_part2_v2 that includes flow along "fake" nodes (blue edges part 1)

df_baseline_part2_v2 = (pd.DataFrame(zip(start_node_output_v2, end_node_output_v2, flow_output_v2, capacity_output_v2, 
                   risk_output_v2), columns = ['Start Node', 'End Node', 'Flow', 'Capacity', 'Risk']))
df_baseline_part2_v2

,Start Node,End Node,Flow,Capacity,Risk
0,1,50,0,8500,0.0
1,2,51,0,40924,0.0
2,3,52,0,35604,0.0
3,4,53,0,49135,0.0
4,5,54,0,44389,0.0
5,6,55,0,1600,0.0
6,7,56,1881,53422,0.0
7,8,57,0,1150,0.0
8,9,58,0,43870,0.0
9,10,59,0,44046,0.0


In [61]:
# Add baseline flow part 2 to df_nodes_final

df_nodes_final = (pd.merge(df_nodes_final, df_baseline_part2_v2, how = 'left', left_on = 'Node ID', 
                           right_on = 'Start Node'))
df_nodes_final = df_nodes_final.drop(['Start Node', 'End Node', 'Capacity', 'Risk'], axis = 1)
df_nodes_final = df_nodes_final.rename(columns = {'Flow': 'Flow - Baseline part 2'})
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Longitude,Total Risks,Flow - Baseline part 1,Net Flow - Baseline part 1,Remaining Storage - Baseline part 1,Remaining Storage Capacity - Baseline part 1,New Supplies - Baseline,New Demand - Baseline,New Supply/Demand - Baseline,Flow - Baseline part 2
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,...,-122.20,0.120,0,0.0,0.0,0.00,0.0,0.00,0.00,0
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,-122.53,0.060,5106,1521.0,0.0,2301.50,0.0,2301.50,-2301.50,0
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,...,-118.53,0.060,14303,0.0,614.0,1881.35,0.0,1881.35,-1881.35,0
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,...,-116.56,0.060,3585,955.0,0.0,2636.00,0.0,2636.00,-2636.00,0
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,...,-107.77,0.060,10350,735.0,0.0,2736.95,0.0,2736.95,-2736.95,0
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,...,-103.28,0.121,14400,-14400.0,0.0,0.00,0.0,0.00,0.00,0
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,...,-109.79,0.060,0,0.0,798.0,1873.10,0.0,1873.10,-1873.10,1881
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,...,-112.16,0.120,10350,-10350.0,0.0,0.00,0.0,0.00,0.00,0
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,...,-112.25,0.060,6230,1944.0,0.0,2505.00,0.0,2505.00,-2505.00,0
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,...,-117.00,0.060,3978,393.0,0.0,2401.20,0.0,2401.20,-2401.20,0


In [62]:
# Create df_baseline_part2_gephi from df_baseline_part2 to format for csv export and gephi import

df_baseline_part2_gephi = df_baseline_part2[df_baseline_part2['Flow'] > 0]
df_baseline_part2_gephi = df_baseline_part2_gephi[['Start Node', 'End Node', 'Flow']]
df_baseline_part2_gephi['Start-End Node'] = (df_baseline_part2_gephi['Start Node'].astype(str) + ' - ' + 
                                             df_baseline_part2_gephi['End Node'].astype(str))
df_baseline_part2_gephi

,Start Node,End Node,Flow,Start-End Node
5,7,3,1881,7 - 3
35,19,7,3754,19 - 7
36,19,14,1406,19 - 14
37,19,22,1213,19 - 22
38,20,19,8708,20 - 19
55,26,40,2564,26 - 40
82,33,37,2869,33 - 37
84,33,38,8414,33 - 38
85,34,33,8380,34 - 33
87,34,42,3819,34 - 42


In [63]:
# Create df_baseline_gephi to format for csv export and gephi import

df_baseline_gephi = pd.merge(df_baseline_part1_gephi, df_baseline_part2_gephi, how = 'outer', on = 'Start-End Node')
df_baseline_gephi = df_baseline_gephi.fillna(0)
df_baseline_gephi['Source'] = df_baseline_gephi[['Start Node_x', 'Start Node_y']].max(axis = 1)
df_baseline_gephi['Target'] = df_baseline_gephi[['End Node_x', 'End Node_y']].max(axis = 1)
df_baseline_gephi['Weight'] = df_baseline_gephi['Flow_x'] + df_baseline_gephi['Flow_y']
df_baseline_gephi = df_baseline_gephi[['Source', 'Target', 'Weight']]
df_baseline_gephi['Type'] = 'Directed'
convert_dict = {'Source': int, 'Target': int, 'Weight': int}
df_baseline_gephi = df_baseline_gephi.astype(convert_dict)
df_baseline_gephi

,Source,Target,Weight,Type
0,2,10,3585,Directed
1,2,11,1521,Directed
2,3,6,14303,Directed
3,4,2,3585,Directed
4,5,8,10350,Directed
5,9,4,2630,Directed
6,9,13,3600,Directed
7,10,12,3978,Directed
8,11,12,2488,Directed
9,14,3,14303,Directed


In [64]:
# Find the flow from each node

df_baseline_part2_start_node_flow = df_baseline_part2[['Start Node', 'Flow']]
df_baseline_part2_start_node_flow = df_baseline_part2_start_node_flow.groupby(['Start Node']).sum()
df_baseline_part2_start_node_flow.reset_index(inplace = True)
df_baseline_part2_start_node_flow

,Start Node,Flow
0,2,0
1,3,0
2,4,0
3,5,0
4,7,1881
5,9,0
6,10,0
7,11,0
8,12,0
9,14,0


In [65]:
# Find the flow to each node

df_baseline_part2_end_node_flow = df_baseline_part2[['End Node', 'Flow']]
df_baseline_part2_end_node_flow = df_baseline_part2_end_node_flow.groupby(['End Node']).sum()
df_baseline_part2_end_node_flow.reset_index(inplace = True)
df_baseline_part2_end_node_flow

,End Node,Flow
0,1,0
1,2,0
2,3,1881
3,4,0
4,5,0
5,6,0
6,7,3754
7,8,0
8,9,0
9,10,0


In [66]:
# Combine flow to/from each node

df_baseline_part2_net_flow = (pd.merge(df_baseline_part2_start_node_flow, df_baseline_part2_end_node_flow, 
                                how = 'outer', left_on = 'Start Node', right_on = 'End Node'))
df_baseline_part2_net_flow = df_baseline_part2_net_flow.fillna(0)
df_baseline_part2_net_flow['Node ID'] = df_baseline_part2_net_flow[['Start Node', 'End Node']].max(axis = 1)
df_baseline_part2_net_flow

,Start Node,Flow_x,End Node,Flow_y,Node ID
0,2.0,0.0,2.0,0.0,2.0
1,3.0,0.0,3.0,1881.0,3.0
2,4.0,0.0,4.0,0.0,4.0
3,5.0,0.0,5.0,0.0,5.0
4,7.0,1881.0,7.0,3754.0,7.0
5,9.0,0.0,9.0,0.0,9.0
6,10.0,0.0,10.0,0.0,10.0
7,11.0,0.0,11.0,0.0,11.0
8,12.0,0.0,12.0,0.0,12.0
9,14.0,0.0,14.0,1406.0,14.0


In [67]:
# Calculate net flow from each node

df_baseline_part2_net_flow['Net Flow - Baseline part 2'] = (df_baseline_part2_net_flow['Flow_x'] - 
                                                            df_baseline_part2_net_flow['Flow_y'])
df_baseline_part2_net_flow = df_baseline_part2_net_flow[['Node ID', 'Net Flow - Baseline part 2']]
df_baseline_part2_net_flow

,Node ID,Net Flow - Baseline part 2
0,2.0,0.0
1,3.0,-1881.0
2,4.0,0.0
3,5.0,0.0
4,7.0,-1873.0
5,9.0,0.0
6,10.0,0.0
7,11.0,0.0
8,12.0,0.0
9,14.0,-1406.0


In [68]:
# Add columns in df_nodes_final and update values

df_nodes_final = pd.merge(df_nodes_final, df_baseline_part2_net_flow, how = 'outer', on = 'Node ID')
df_nodes_final['Total Flow - Baseline'] = (df_nodes_final['Flow - Baseline part 1'] + 
                                           df_nodes_final['Flow - Baseline part 2'])
df_nodes_final['Remaining Storage - Baseline'] = (df_nodes_final['Remaining Storage - Baseline part 1'] - 
                                                  df_nodes_final['Net Flow - Baseline part 2'])
df_nodes_final.loc[df_nodes_final['Storage Capacity'] == 0, 'Remaining Storage - Baseline'] = 0
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Net Flow - Baseline part 1,Remaining Storage - Baseline part 1,Remaining Storage Capacity - Baseline part 1,New Supplies - Baseline,New Demand - Baseline,New Supply/Demand - Baseline,Flow - Baseline part 2,Net Flow - Baseline part 2,Total Flow - Baseline,Remaining Storage - Baseline
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,...,0.0,0.0,0.00,0.0,0.00,0.00,0,0.0,0,0.0
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,1521.0,0.0,2301.50,0.0,2301.50,-2301.50,0,0.0,5106,0.0
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,...,0.0,614.0,1881.35,0.0,1881.35,-1881.35,0,-1881.0,14303,2495.0
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,...,955.0,0.0,2636.00,0.0,2636.00,-2636.00,0,0.0,3585,0.0
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,...,735.0,0.0,2736.95,0.0,2736.95,-2736.95,0,0.0,10350,0.0
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,...,-14400.0,0.0,0.00,0.0,0.00,0.00,0,0.0,14400,0.0
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,...,0.0,798.0,1873.10,0.0,1873.10,-1873.10,1881,-1873.0,1881,2671.0
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,...,-10350.0,0.0,0.00,0.0,0.00,0.00,0,0.0,10350,0.0
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,...,1944.0,0.0,2505.00,0.0,2505.00,-2505.00,0,0.0,6230,0.0
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,...,393.0,0.0,2401.20,0.0,2401.20,-2401.20,0,0.0,3978,0.0


In [69]:
# Scenario 1 - part 1: Max flow with min cost using actual capacity

flow = pywrapgraph.SimpleMinCostFlow()

# Add each arc
for i in range(len(start_nodes)):
    flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i], capacity[i], risk[i])

# Add node supplies
for i in range(len(supplies)):
    flow.SetNodeSupply(int(id_supplies[i]), supplies[i])

remaining_capacity = []
    
start_node_output = []
end_node_output = []
flow_output = []
capacity_output = []
risk_output = []

start_node_output_v2 = []
end_node_output_v2 = []
flow_output_v2 = []
capacity_output_v2 = []
risk_output_v2 = []

# Find the minimum cost flow
if flow.SolveMaxFlowWithMinCost():
    for i in range(flow.NumArcs()):
        remaining_capacity.append(capacity[i] - flow.Flow(i))
        if start_nodes[i] > nodes_len:
            start_node_output.append(flow.Tail(i))
            end_node_output.append(flow.Head(i))
            flow_output.append(flow.Flow(i))
            capacity_output.append(flow.Capacity(i))
            risk_calc = (flow.UnitCost(i)/1000)
            risk_output.append(risk_calc)
        else:
            start_node_output_v2.append(flow.Tail(i))
            end_node_output_v2.append(flow.Head(i))
            flow_output_v2.append(flow.Flow(i))
            capacity_output_v2.append(flow.Capacity(i))
            risk_calc = (flow.UnitCost(i)/1000)
            risk_output_v2.append(risk_calc)
    
    total_flow_output_scenario1_part1 = sum(flow_output) + sum(flow_output_v2)
    accum_risk_scenario1_part1 = float(flow.OptimalCost()) / 1000

# Create df_scenario1_part1 that includes flow along "true" edges (black edges part 2)
df_scenario1_part1 = (pd.DataFrame(zip(start_node_output, end_node_output, flow_output, capacity_output, risk_output), 
                             columns = ['Start Node', 'End Node', 'Flow', 'Capacity', 'Risk']))
df_scenario1_part1['Start Node'] = df_scenario1_part1['Start Node'] - nodes_len
df_scenario1_part1['Edge ID'] = df_scenario1_part1.index + 1
df_scenario1_part1.tail(50)

,Start Node,End Node,Flow,Capacity,Risk,Edge ID
73,30,45,14400,47068,0.452,74
74,31,28,7650,73800,0.400,75
75,31,28,0,65539,0.480,76
76,31,29,0,66176,0.970,77
77,31,29,0,78039,0.740,78
78,31,32,0,69864,0.400,79
79,31,44,13050,59292,0.452,80
80,32,28,744,68852,0.140,81
81,32,33,0,67678,0.140,82
82,33,37,0,23723,0.140,83


In [70]:
# Create df_scenario1_v2 that includes flow along "fake" nodes (blue edges part 1)

df_scenario1_part1_v2 = (pd.DataFrame(zip(start_node_output_v2, end_node_output_v2, flow_output_v2, capacity_output_v2, 
                   risk_output_v2), columns = ['Start Node', 'End Node', 'Flow', 'Capacity', 'Risk']))
df_scenario1_part1_v2

,Start Node,End Node,Flow,Capacity,Risk
0,1,50,0,8500,0.120
1,2,51,5106,46030,0.060
2,3,52,2030,49907,0.060
3,4,53,3585,52720,0.060
4,5,54,735,54739,0.060
5,6,55,14400,16000,0.121
6,7,56,4225,53422,0.060
7,8,57,10350,11500,0.120
8,9,58,15845,50100,0.060
9,10,59,393,48024,0.060


In [71]:
# Add scenario 1 flow to df_nodes_final

df_nodes_final = (pd.merge(df_nodes_final, df_scenario1_part1_v2, how = 'left', left_on = 'Node ID', 
                           right_on = 'Start Node'))
df_nodes_final = df_nodes_final.drop(['Start Node', 'End Node', 'Capacity', 'Risk'], axis = 1)
df_nodes_final = df_nodes_final.rename(columns = {'Flow': 'Flow - Scenario 1 part 1'})
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Remaining Storage - Baseline part 1,Remaining Storage Capacity - Baseline part 1,New Supplies - Baseline,New Demand - Baseline,New Supply/Demand - Baseline,Flow - Baseline part 2,Net Flow - Baseline part 2,Total Flow - Baseline,Remaining Storage - Baseline,Flow - Scenario 1 part 1
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,...,0.0,0.00,0.0,0.00,0.00,0,0.0,0,0.0,0
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,0.0,2301.50,0.0,2301.50,-2301.50,0,0.0,5106,0.0,5106
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,...,614.0,1881.35,0.0,1881.35,-1881.35,0,-1881.0,14303,2495.0,2030
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,...,0.0,2636.00,0.0,2636.00,-2636.00,0,0.0,3585,0.0,3585
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,...,0.0,2736.95,0.0,2736.95,-2736.95,0,0.0,10350,0.0,735
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,...,0.0,0.00,0.0,0.00,0.00,0,0.0,14400,0.0,14400
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,...,798.0,1873.10,0.0,1873.10,-1873.10,1881,-1873.0,1881,2671.0,4225
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,...,0.0,0.00,0.0,0.00,0.00,0,0.0,10350,0.0,10350
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,...,0.0,2505.00,0.0,2505.00,-2505.00,0,0.0,6230,0.0,15845
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,...,0.0,2401.20,0.0,2401.20,-2401.20,0,0.0,3978,0.0,393


In [72]:
# Create df_scenario1_part1_gephi from df_scenario1_part1 to format for csv export and gephi import

df_scenario1_part1_gephi = df_scenario1_part1[df_scenario1_part1['Flow'] > 0]
df_scenario1_part1_gephi = df_scenario1_part1_gephi[['Start Node', 'End Node', 'Flow']]
df_scenario1_part1_gephi['Start-End Node'] = (df_scenario1_part1_gephi['Start Node'].astype(str) + ' - ' + 
                                              df_scenario1_part1_gephi['End Node'].astype(str))
df_scenario1_part1_gephi

,Start Node,End Node,Flow,Start-End Node
1,2,11,5106,2 - 11
2,3,6,2030,3 - 6
3,4,2,3585,4 - 2
4,5,8,735,5 - 8
6,7,6,4225,7 - 6
8,9,4,2630,9 - 4
9,9,8,9615,9 - 8
10,9,13,3600,9 - 13
13,10,12,393,10 - 12
15,11,12,6073,11 - 12


In [73]:
# Find the flow from each node

df_scenario1_part1_start_node_flow = df_scenario1_part1[['Start Node', 'Flow']]
df_scenario1_part1_start_node_flow = df_scenario1_part1_start_node_flow.groupby(['Start Node']).sum()
df_scenario1_part1_start_node_flow.reset_index(inplace = True)
df_scenario1_part1_start_node_flow

,Start Node,Flow
0,2,5106
1,3,2030
2,4,3585
3,5,735
4,7,4225
5,9,15845
6,10,393
7,11,6073
8,12,0
9,14,1416


In [74]:
# Find the flow to each node

df_scenario1_part1_end_node_flow = df_scenario1_part1[['End Node', 'Flow']]
df_scenario1_part1_end_node_flow = df_scenario1_part1_end_node_flow.groupby(['End Node']).sum()
df_scenario1_part1_end_node_flow.reset_index(inplace = True)
df_scenario1_part1_end_node_flow

,End Node,Flow
0,1,0
1,2,3585
2,3,1416
3,4,2630
4,5,0
5,6,14400
6,7,3427
7,8,10350
8,9,13901
9,10,0


In [75]:
# Combine flow to/from each node

df_scenario1_part1_net_flow = (pd.merge(df_scenario1_part1_start_node_flow, df_scenario1_part1_end_node_flow, 
                               how = 'outer', left_on = 'Start Node', right_on = 'End Node'))
df_scenario1_part1_net_flow = df_scenario1_part1_net_flow.fillna(0)
df_scenario1_part1_net_flow['Node ID'] = df_scenario1_part1_net_flow[['Start Node', 'End Node']].max(axis = 1)
df_scenario1_part1_net_flow

,Start Node,Flow_x,End Node,Flow_y,Node ID
0,2.0,5106.0,2.0,3585.0,2.0
1,3.0,2030.0,3.0,1416.0,3.0
2,4.0,3585.0,4.0,2630.0,4.0
3,5.0,735.0,5.0,0.0,5.0
4,7.0,4225.0,7.0,3427.0,7.0
5,9.0,15845.0,9.0,13901.0,9.0
6,10.0,393.0,10.0,0.0,10.0
7,11.0,6073.0,11.0,5106.0,11.0
8,12.0,0.0,12.0,13405.0,12.0
9,14.0,1416.0,14.0,0.0,14.0


In [76]:
# Calculate net flow from each node

df_scenario1_part1_net_flow['Net Flow - Scenario 1 part 1'] = (df_scenario1_part1_net_flow['Flow_x'] - 
                                                               df_scenario1_part1_net_flow['Flow_y'])
df_scenario1_part1_net_flow = df_scenario1_part1_net_flow[['Node ID', 'Net Flow - Scenario 1 part 1']]
df_scenario1_part1_net_flow

,Node ID,Net Flow - Scenario 1 part 1
0,2.0,1521.0
1,3.0,614.0
2,4.0,955.0
3,5.0,735.0
4,7.0,798.0
5,9.0,1944.0
6,10.0,393.0
7,11.0,967.0
8,12.0,-13405.0
9,14.0,1416.0


In [77]:
# Add columns in df_nodes_final and update values

df_nodes_final = pd.merge(df_nodes_final, df_scenario1_part1_net_flow, how = 'outer', on = 'Node ID')
df_nodes_final['Remaining Storage - Scenario 1 part 1'] = (df_nodes_final['Current Storage'] - 
                                                    df_nodes_final['Net Flow - Scenario 1 part 1'])
df_nodes_final.loc[df_nodes_final['Remaining Storage - Scenario 1 part 1'] < 0, 'Remaining Storage - Scenario 1 part 1'] = 0
df_nodes_final.loc[df_nodes_final['Demand'] > 0, 'Remaining Storage - Scenario 1 part 1'] = 0
df_nodes_final.loc[df_nodes_final['Demand'] > 0, 'Remaining Storage Capacity - Scenario 1 part 1'] = 0
df_nodes_final['Remaining Storage Capacity - Scenario 1 part 1'] = (df_nodes_final['Storage Capacity'] - 
                                                             df_nodes_final['Remaining Storage - Scenario 1 part 1'])
df_nodes_final['New Supplies - Scenario 1'] = (df_nodes_final['Supply'] - 
                                               df_nodes_final['Net Flow - Scenario 1 part 1'])
df_nodes_final.loc[df_nodes_final['New Supplies - Scenario 1'] < 0, 'New Supplies - Scenario 1'] = 0
df_nodes_final.loc[df_nodes_final['Resupply'] == 'No', 'New Supplies - Scenario 1'] = 0
df_nodes_final['New Demand - Scenario 1'] = df_nodes_final['Remaining Storage Capacity - Scenario 1 part 1']
df_nodes_final['New Supply/Demand - Scenario 1'] = (df_nodes_final['New Supplies - Scenario 1'] - 
                                                    df_nodes_final['New Demand - Scenario 1'])
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Net Flow - Baseline part 2,Total Flow - Baseline,Remaining Storage - Baseline,Flow - Scenario 1 part 1,Net Flow - Scenario 1 part 1,Remaining Storage - Scenario 1 part 1,Remaining Storage Capacity - Scenario 1 part 1,New Supplies - Scenario 1,New Demand - Scenario 1,New Supply/Demand - Scenario 1
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,...,0.0,0,0.0,0,0.0,0.0,0.00,0.0,0.00,0.00
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,0.0,5106,0.0,5106,1521.0,0.0,2301.50,0.0,2301.50,-2301.50
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,...,-1881.0,14303,2495.0,2030,614.0,0.0,2495.35,0.0,2495.35,-2495.35
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,...,0.0,3585,0.0,3585,955.0,0.0,2636.00,0.0,2636.00,-2636.00
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,...,0.0,10350,0.0,735,735.0,0.0,2736.95,0.0,2736.95,-2736.95
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,...,0.0,14400,0.0,14400,-14400.0,0.0,0.00,0.0,0.00,0.00
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,...,-1873.0,1881,2671.0,4225,798.0,0.0,2671.10,0.0,2671.10,-2671.10
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,...,0.0,10350,0.0,10350,-10350.0,0.0,0.00,0.0,0.00,0.00
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,...,0.0,6230,0.0,15845,1944.0,0.0,2505.00,0.0,2505.00,-2505.00
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,...,0.0,3978,0.0,393,393.0,0.0,2401.20,0.0,2401.20,-2401.20


In [78]:
# Create df_new_supplies to capture supply from all new nodes (blue nodes part 2)

df_new_supplies = df_nodes_final.copy()
df_new_supplies = df_new_supplies[['Node ID', 'New Supply/Demand - Scenario 1']]
df_new_supplies['Node ID'] += nodes_len
df_new_supplies['New Supply/Demand - Scenario 1'] = 0
df_new_supplies

,Node ID,New Supply/Demand - Scenario 1
0,50,0
1,51,0
2,52,0
3,53,0
4,54,0
5,55,0
6,56,0
7,57,0
8,58,0
9,59,0


In [79]:
# Create df_new_supplies_all to find supply from all black (part 1) and blue (part 2) nodes

df_new_supplies_all = pd.concat([df_nodes_final, df_new_supplies])
df_new_supplies_all = df_new_supplies_all.reset_index(drop = True)
df_new_supplies_all

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Net Flow - Baseline part 2,Total Flow - Baseline,Remaining Storage - Baseline,Flow - Scenario 1 part 1,Net Flow - Scenario 1 part 1,Remaining Storage - Scenario 1 part 1,Remaining Storage Capacity - Scenario 1 part 1,New Supplies - Scenario 1,New Demand - Scenario 1,New Supply/Demand - Scenario 1
0,1,Washington AFB,End,Use,8500.0,0.00,0.0,7650.0,0.0,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030.0,2301.50,0.0,0.0,1521.0,No,...,0.0,5106.0,0.0,5106.0,1521.0,0.0,2301.50,0.0,2301.50,-2301.50
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907.0,2495.35,0.0,0.0,614.0,No,...,-1881.0,14303.0,2495.0,2030.0,614.0,0.0,2495.35,0.0,2495.35,-2495.35
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720.0,2636.00,0.0,0.0,955.0,No,...,0.0,3585.0,0.0,3585.0,955.0,0.0,2636.00,0.0,2636.00,-2636.00
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739.0,2736.95,0.0,0.0,735.0,No,...,0.0,10350.0,0.0,735.0,735.0,0.0,2736.95,0.0,2736.95,-2736.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
94,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
95,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
96,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00


In [80]:
# Create empty lists to use in the second min cost flow calculation

new_supplies = []
new_id_supplies = []

In [81]:
# Append lists in the correct format

for i in range(len(df_new_supplies_all)):
    new_supplies.append(int(df_new_supplies_all.loc[i, 'New Supply/Demand - Scenario 1']))
    new_id_supplies.append(int(df_new_supplies_all.loc[i, 'Node ID']))

In [82]:
# Create df_new_inputs in order to create lists for second min cost flow calculation

list = ({'Start Nodes': start_nodes, 'End Nodes': end_nodes, 'Remaining Capacity': remaining_capacity, 
               'Risk': risk})
df_new_inputs = pd.DataFrame(list)
df_new_inputs

,Start Nodes,End Nodes,Remaining Capacity,Risk
0,51,10,65627,219
1,51,11,59057,140
2,52,6,35745,450
3,53,2,49826,140
4,54,8,24702,219
...,...,...,...,...
167,45,94,1600,120
168,46,95,53017,90
169,47,96,1500,120
170,48,97,1800,120


In [83]:
# Create empty lists to use in the second min cost flow calculation

start_nodes_2 = []
end_nodes_2 = []
remaining_capacity_2 = []
risk_2 = []

In [84]:
# Append lists in the correct format

for i in range(len(df_new_inputs)):
    start_nodes_2.append(int(df_new_inputs.loc[i, 'Start Nodes']))
    end_nodes_2.append(int(df_new_inputs.loc[i, 'End Nodes']))
    remaining_capacity_2.append(int(float(df_new_inputs.loc[i, 'Remaining Capacity'])))
    risk_2.append(int(float(df_new_inputs.loc[i, 'Risk'])))

In [85]:
# Scenario 1 - part 2: Max flow with min cost using actual capacity

flow = pywrapgraph.SimpleMinCostFlow()

# Add each arc
for i in range(len(start_nodes)):
    flow.AddArcWithCapacityAndUnitCost(start_nodes_2[i], end_nodes_2[i], remaining_capacity_2[i], risk_2[i])

# Add node supplies
for i in range(len(new_supplies)):
    flow.SetNodeSupply(int(new_id_supplies[i]), new_supplies[i])
    
start_node_output = []
end_node_output = []
flow_output = []
capacity_output = []
risk_output = []

start_node_output_v2 = []
end_node_output_v2 = []
flow_output_v2 = []
capacity_output_v2 = []
risk_output_v2 = []

# Find the minimum cost flow
if flow.SolveMaxFlowWithMinCost():
    for i in range(flow.NumArcs()):
        if start_nodes[i] > nodes_len:
            start_node_output.append(flow.Tail(i))
            end_node_output.append(flow.Head(i))
            flow_output.append(flow.Flow(i))
            capacity_output.append(flow.Capacity(i))
            risk_calc = (flow.UnitCost(i)/1000)
            risk_output.append(risk_calc)
        else:
            start_node_output_v2.append(flow.Tail(i))
            end_node_output_v2.append(flow.Head(i))
            flow_output_v2.append(flow.Flow(i))
            capacity_output_v2.append(flow.Capacity(i))
            risk_calc = (flow.UnitCost(i)/1000)
            risk_output_v2.append(risk_calc)
    
    total_flow_output_scenario1_part2 = sum(flow_output) + sum(flow_output_v2)
    total_flow_output_scenario1 = total_flow_output_scenario1_part1 + total_flow_output_scenario1_part2
    accum_risk_scenario1_part2 = float(flow.OptimalCost()) / 1000
    accum_risk_scenario1 = accum_risk_scenario1_part1 + accum_risk_scenario1_part2
    accum_risk_percent_scenario1 = accum_risk_scenario1 / total_flow_output_scenario1

# Create df_scenario1_part2 that includes flow along "true" edges (black edges part 2)
df_scenario1_part2 = (pd.DataFrame(zip(start_node_output, end_node_output, flow_output, capacity_output, risk_output), 
                             columns = ['Start Node', 'End Node', 'Flow', 'Capacity', 'Risk']))
df_scenario1_part2['Start Node'] = df_scenario1_part2['Start Node'] - nodes_len
df_scenario1_part2['Edge ID'] = df_scenario1_part2.index + 1
df_scenario1_part2.tail(50)

,Start Node,End Node,Flow,Capacity,Risk,Edge ID
73,30,45,0,32668,0.452,74
74,31,28,0,66150,0.400,75
75,31,28,0,65539,0.480,76
76,31,29,0,66176,0.970,77
77,31,29,0,78039,0.740,78
78,31,32,0,69864,0.400,79
79,31,44,0,46242,0.452,80
80,32,28,0,68108,0.140,81
81,32,33,0,67678,0.140,82
82,33,37,3386,23723,0.140,83


In [86]:
# Create df_scenario1_part2_v2 that includes flow along "fake" nodes (blue edges part 1)

df_scenario1_part2_v2 = (pd.DataFrame(zip(start_node_output_v2, end_node_output_v2, flow_output_v2, capacity_output_v2, 
                   risk_output_v2), columns = ['Start Node', 'End Node', 'Flow', 'Capacity', 'Risk']))
df_scenario1_part2_v2

,Start Node,End Node,Flow,Capacity,Risk
0,1,50,0,8500,0.120
1,2,51,0,40924,0.060
2,3,52,0,47877,0.060
3,4,53,0,49135,0.060
4,5,54,0,54004,0.060
5,6,55,0,1600,0.121
6,7,56,2495,49197,0.060
7,8,57,0,1150,0.120
8,9,58,0,34255,0.060
9,10,59,0,47631,0.060


In [87]:
# Add scenario 1 flow part 2 to df_nodes_final

df_nodes_final = (pd.merge(df_nodes_final, df_scenario1_part2_v2, how = 'left', left_on = 'Node ID', 
                           right_on = 'Start Node'))
df_nodes_final = df_nodes_final.drop(['Start Node', 'End Node', 'Capacity', 'Risk'], axis = 1)
df_nodes_final = df_nodes_final.rename(columns = {'Flow': 'Flow - Scenario 1 part 2'})
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Total Flow - Baseline,Remaining Storage - Baseline,Flow - Scenario 1 part 1,Net Flow - Scenario 1 part 1,Remaining Storage - Scenario 1 part 1,Remaining Storage Capacity - Scenario 1 part 1,New Supplies - Scenario 1,New Demand - Scenario 1,New Supply/Demand - Scenario 1,Flow - Scenario 1 part 2
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,...,0,0.0,0,0.0,0.0,0.00,0.0,0.00,0.00,0
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,5106,0.0,5106,1521.0,0.0,2301.50,0.0,2301.50,-2301.50,0
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,...,14303,2495.0,2030,614.0,0.0,2495.35,0.0,2495.35,-2495.35,0
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,...,3585,0.0,3585,955.0,0.0,2636.00,0.0,2636.00,-2636.00,0
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,...,10350,0.0,735,735.0,0.0,2736.95,0.0,2736.95,-2736.95,0
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,...,14400,0.0,14400,-14400.0,0.0,0.00,0.0,0.00,0.00,0
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,...,1881,2671.0,4225,798.0,0.0,2671.10,0.0,2671.10,-2671.10,2495
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,...,10350,0.0,10350,-10350.0,0.0,0.00,0.0,0.00,0.00,0
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,...,6230,0.0,15845,1944.0,0.0,2505.00,0.0,2505.00,-2505.00,0
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,...,3978,0.0,393,393.0,0.0,2401.20,0.0,2401.20,-2401.20,0


In [88]:
# Create df_scenario1_part2_gephi from df_scenario1_part2 to format for csv export and gephi import

df_scenario1_part2_gephi = df_scenario1_part2[df_scenario1_part2['Flow'] > 0]
df_scenario1_part2_gephi = df_scenario1_part2_gephi[['Start Node', 'End Node', 'Flow']]
df_scenario1_part2_gephi['Start-End Node'] = (df_scenario1_part2_gephi['Start Node'].astype(str) + ' - ' + 
                                              df_scenario1_part2_gephi['End Node'].astype(str))
df_scenario1_part2_gephi

,Start Node,End Node,Flow,Start-End Node
5,7,3,2495,7 - 3
35,19,7,5166,19 - 7
36,19,14,2295,19 - 14
37,19,22,2533,19 - 22
38,20,19,12426,20 - 19
82,33,37,3386,33 - 37
84,33,38,9842,33 - 38
85,34,33,7199,34 - 33
87,34,42,7211,34 - 42
89,35,33,8506,35 - 33


In [89]:
# Create df_scenario1_gephi to format for csv export and gephi import

df_scenario1_gephi = pd.merge(df_scenario1_part1_gephi, df_scenario1_part2_gephi, how = 'outer', on = 'Start-End Node')
df_scenario1_gephi = df_scenario1_gephi.fillna(0)
df_scenario1_gephi['Source'] = df_scenario1_gephi[['Start Node_x', 'Start Node_y']].max(axis = 1)
df_scenario1_gephi['Target'] = df_scenario1_gephi[['End Node_x', 'End Node_y']].max(axis = 1)
df_scenario1_gephi['Weight'] = df_scenario1_gephi['Flow_x'] + df_scenario1_gephi['Flow_y']
df_scenario1_gephi = df_scenario1_gephi[['Source', 'Target', 'Weight']]
df_scenario1_gephi['Type'] = 'Directed'
convert_dict = {'Source': int, 'Target': int, 'Weight': int}
df_scenario1_gephi = df_scenario1_gephi.astype(convert_dict)
df_scenario1_gephi

,Source,Target,Weight,Type
0,2,11,5106,Directed
1,3,6,2030,Directed
2,4,2,3585,Directed
3,5,8,735,Directed
4,7,6,4225,Directed
5,9,4,2630,Directed
6,9,8,9615,Directed
7,9,13,3600,Directed
8,10,12,393,Directed
9,11,12,6073,Directed


In [90]:
# Find the flow from each node

df_scenario1_part2_start_node_flow = df_scenario1_part2[['Start Node', 'Flow']]
df_scenario1_part2_start_node_flow = df_scenario1_part2_start_node_flow.groupby(['Start Node']).sum()
df_scenario1_part2_start_node_flow.reset_index(inplace = True)
df_scenario1_part2_start_node_flow

,Start Node,Flow
0,2,0
1,3,0
2,4,0
3,5,0
4,7,2495
5,9,0
6,10,0
7,11,0
8,12,0
9,14,0


In [91]:
# Find the flow to each node

df_scenario1_part2_end_node_flow = df_scenario1_part2[['End Node', 'Flow']]
df_scenario1_part2_end_node_flow = df_scenario1_part2_end_node_flow.groupby(['End Node']).sum()
df_scenario1_part2_end_node_flow.reset_index(inplace = True)
df_scenario1_part2_end_node_flow

,End Node,Flow
0,1,0
1,2,0
2,3,2495
3,4,0
4,5,0
5,6,0
6,7,5166
7,8,0
8,9,0
9,10,0


In [92]:
# Combine flow to/from each node

df_scenario1_part2_net_flow = (pd.merge(df_scenario1_part2_start_node_flow, df_scenario1_part2_end_node_flow, 
                                how = 'outer', left_on = 'Start Node', right_on = 'End Node'))
df_scenario1_part2_net_flow = df_scenario1_part2_net_flow.fillna(0)
df_scenario1_part2_net_flow['Node ID'] = df_scenario1_part2_net_flow[['Start Node', 'End Node']].max(axis = 1)
df_scenario1_part2_net_flow

,Start Node,Flow_x,End Node,Flow_y,Node ID
0,2.0,0.0,2.0,0.0,2.0
1,3.0,0.0,3.0,2495.0,3.0
2,4.0,0.0,4.0,0.0,4.0
3,5.0,0.0,5.0,0.0,5.0
4,7.0,2495.0,7.0,5166.0,7.0
5,9.0,0.0,9.0,0.0,9.0
6,10.0,0.0,10.0,0.0,10.0
7,11.0,0.0,11.0,0.0,11.0
8,12.0,0.0,12.0,0.0,12.0
9,14.0,0.0,14.0,2295.0,14.0


In [93]:
# Calculate net flow from each node

df_scenario1_part2_net_flow['Net Flow - Scenario 1 part 2'] = (df_scenario1_part2_net_flow['Flow_x'] - 
                                                               df_scenario1_part2_net_flow['Flow_y'])
df_scenario1_part2_net_flow = df_scenario1_part2_net_flow[['Node ID', 'Net Flow - Scenario 1 part 2']]
df_scenario1_part2_net_flow

,Node ID,Net Flow - Scenario 1 part 2
0,2.0,0.0
1,3.0,-2495.0
2,4.0,0.0
3,5.0,0.0
4,7.0,-2671.0
5,9.0,0.0
6,10.0,0.0
7,11.0,0.0
8,12.0,0.0
9,14.0,-2295.0


In [94]:
# Add columns in df_nodes_final and update values

df_nodes_final = pd.merge(df_nodes_final, df_scenario1_part2_net_flow, how = 'outer', on = 'Node ID')
df_nodes_final['Total Flow - Scenario 1'] = (df_nodes_final['Flow - Scenario 1 part 1'] + 
                                             df_nodes_final['Flow - Scenario 1 part 2'])
df_nodes_final['Remaining Storage - Scenario 1'] = (df_nodes_final['Remaining Storage - Scenario 1 part 1'] - 
                                                    df_nodes_final['Net Flow - Scenario 1 part 2'])
df_nodes_final.loc[df_nodes_final['Storage Capacity'] == 0, 'Remaining Storage - Scenario 1'] = 0
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Net Flow - Scenario 1 part 1,Remaining Storage - Scenario 1 part 1,Remaining Storage Capacity - Scenario 1 part 1,New Supplies - Scenario 1,New Demand - Scenario 1,New Supply/Demand - Scenario 1,Flow - Scenario 1 part 2,Net Flow - Scenario 1 part 2,Total Flow - Scenario 1,Remaining Storage - Scenario 1
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,...,0.0,0.0,0.00,0.0,0.00,0.00,0,0.0,0,0.0
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,1521.0,0.0,2301.50,0.0,2301.50,-2301.50,0,0.0,5106,0.0
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,...,614.0,0.0,2495.35,0.0,2495.35,-2495.35,0,-2495.0,2030,2495.0
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,...,955.0,0.0,2636.00,0.0,2636.00,-2636.00,0,0.0,3585,0.0
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,...,735.0,0.0,2736.95,0.0,2736.95,-2736.95,0,0.0,735,0.0
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,...,-14400.0,0.0,0.00,0.0,0.00,0.00,0,0.0,14400,0.0
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,...,798.0,0.0,2671.10,0.0,2671.10,-2671.10,2495,-2671.0,6720,2671.0
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,...,-10350.0,0.0,0.00,0.0,0.00,0.00,0,0.0,10350,0.0
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,...,1944.0,0.0,2505.00,0.0,2505.00,-2505.00,0,0.0,15845,0.0
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,...,393.0,0.0,2401.20,0.0,2401.20,-2401.20,0,0.0,393,0.0


In [95]:
# Scenario 2 - part 1: Max flow with min cost using expected capacity

flow = pywrapgraph.SimpleMinCostFlow()

# Add each arc
for i in range(len(start_nodes)):
    flow.AddArcWithCapacityAndUnitCost(start_nodes[i], end_nodes[i], expected_capacity[i], risk[i])
    
# Add node supplies
for i in range(len(supplies)):
    flow.SetNodeSupply(int(id_supplies[i]), supplies[i])
    
start_node_output = []
end_node_output = []
flow_output = []
capacity_output = []
risk_output = []

start_node_output_v2 = []
end_node_output_v2 = []
flow_output_v2 = []
capacity_output_v2 = []
risk_output_v2 = []

# Find the minimum cost flow
if flow.SolveMaxFlowWithMinCost(): 
    for i in range(flow.NumArcs()):
        remaining_expected_capacity.append(capacity[i] - flow.Flow(i))
        if start_nodes[i] > nodes_len:
            start_node_output.append(flow.Tail(i))
            end_node_output.append(flow.Head(i))
            flow_output.append(flow.Flow(i))
            capacity_output.append(flow.Capacity(i))
            risk_calc = (flow.UnitCost(i)/1000)
            risk_output.append(risk_calc)
        else:
            start_node_output_v2.append(flow.Tail(i))
            end_node_output_v2.append(flow.Head(i))
            flow_output_v2.append(flow.Flow(i))
            capacity_output_v2.append(flow.Capacity(i))
            risk_calc = (flow.UnitCost(i)/1000)
            risk_output_v2.append(risk_calc)
    
    total_flow_output_scenario2_part1 = sum(flow_output) + sum(flow_output_v2)
    accum_risk_scenario2_part1 = float(flow.OptimalCost()) / 1000

# Create df_scenario2_part1 that includes flow along "true" nodes (black edges part 2)
df_scenario2_part1 = (pd.DataFrame(zip(start_node_output, end_node_output, flow_output, capacity_output, risk_output), 
                             columns = ['Start Node', 'End Node', 'Flow', 'Capacity', 'Risk']))
df_scenario2_part1['Start Node'] = df_scenario2_part1['Start Node'] - nodes_len
df_scenario2_part1['Edge ID'] = df_scenario2_part1.index + 1
df_scenario2_part1

,Start Node,End Node,Flow,Capacity,Risk,Edge ID
0,2,10,0,51189,0.219,1
1,2,11,2476,55180,0.140,2
2,3,6,2030,20776,0.450,3
3,4,2,955,45933,0.140,4
4,5,8,735,19840,0.219,5
...,...,...,...,...,...,...
118,46,48,0,54296,0.249,119
119,46,48,0,24190,0.480,120
120,47,48,0,17507,0.140,121
121,47,48,0,51409,0.219,122


In [96]:
# Create df_scenario2_part1_v2 that includes flow along "fake" nodes (blue edges part 1)

df_scenario2_part1_v2 = (pd.DataFrame(zip(start_node_output_v2, end_node_output_v2, flow_output_v2, capacity_output_v2, 
                   risk_output_v2), columns = ['Start Node', 'End Node', 'Flow', 'Capacity', 'Risk']))
df_scenario2_part1_v2

,Start Node,End Node,Flow,Capacity,Risk
0,1,50,0,7480,0.120
1,2,51,2476,43268,0.060
2,3,52,2030,46912,0.060
3,4,53,955,49556,0.060
4,5,54,735,51454,0.060
5,6,55,14064,14064,0.121
6,7,56,3889,50216,0.060
7,8,57,8751,10120,0.120
8,9,58,11143,47094,0.060
9,10,59,393,45142,0.060


In [97]:
# Add scenario 2 flow to df_nodes_final

df_nodes_final = (pd.merge(df_nodes_final, df_scenario2_part1_v2, how = 'left', left_on = 'Node ID', 
                           right_on = 'Start Node'))
df_nodes_final = df_nodes_final.drop(['Start Node', 'End Node', 'Capacity', 'Risk'], axis = 1)
df_nodes_final = df_nodes_final.rename(columns = {'Flow': 'Flow - Scenario 2 part 1'})
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Remaining Storage - Scenario 1 part 1,Remaining Storage Capacity - Scenario 1 part 1,New Supplies - Scenario 1,New Demand - Scenario 1,New Supply/Demand - Scenario 1,Flow - Scenario 1 part 2,Net Flow - Scenario 1 part 2,Total Flow - Scenario 1,Remaining Storage - Scenario 1,Flow - Scenario 2 part 1
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,...,0.0,0.00,0.0,0.00,0.00,0,0.0,0,0.0,0
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,0.0,2301.50,0.0,2301.50,-2301.50,0,0.0,5106,0.0,2476
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,...,0.0,2495.35,0.0,2495.35,-2495.35,0,-2495.0,2030,2495.0,2030
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,...,0.0,2636.00,0.0,2636.00,-2636.00,0,0.0,3585,0.0,955
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,...,0.0,2736.95,0.0,2736.95,-2736.95,0,0.0,735,0.0,735
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,...,0.0,0.00,0.0,0.00,0.00,0,0.0,14400,0.0,14064
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,...,0.0,2671.10,0.0,2671.10,-2671.10,2495,-2671.0,6720,2671.0,3889
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,...,0.0,0.00,0.0,0.00,0.00,0,0.0,10350,0.0,8751
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,...,0.0,2505.00,0.0,2505.00,-2505.00,0,0.0,15845,0.0,11143
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,...,0.0,2401.20,0.0,2401.20,-2401.20,0,0.0,393,0.0,393


In [98]:
# Create df_scenario2_part1_gephi from df_scenario2_part1 to format for csv export and gephi import

df_scenario2_part1_gephi = df_scenario2_part1[df_scenario2_part1['Flow'] > 0]
df_scenario2_part1_gephi = df_scenario2_part1_gephi[['Start Node', 'End Node', 'Flow']]
df_scenario2_part1_gephi['Start-End Node'] = (df_scenario2_part1_gephi['Start Node'].astype(str) + ' - ' + 
                                              df_scenario2_part1_gephi['End Node'].astype(str))
df_scenario2_part1_gephi

,Start Node,End Node,Flow,Start-End Node
1,2,11,2476,2 - 11
2,3,6,2030,3 - 6
3,4,2,955,4 - 2
4,5,8,735,5 - 8
6,7,6,3889,7 - 6
9,9,8,7623,9 - 8
10,9,13,3520,9 - 13
11,10,8,393,10 - 8
15,11,12,3443,11 - 12
18,14,3,1416,14 - 3


In [99]:
# Find the flow from each node

df_scenario2_part1_start_node_flow = df_scenario2_part1[['Start Node', 'Flow']]
df_scenario2_part1_start_node_flow = df_scenario2_part1_start_node_flow.groupby(['Start Node']).sum()
df_scenario2_part1_start_node_flow.reset_index(inplace = True)
df_scenario2_part1_start_node_flow

,Start Node,Flow
0,2,2476
1,3,2030
2,4,955
3,5,735
4,7,3889
5,9,11143
6,10,393
7,11,3443
8,12,0
9,14,1416


In [100]:
# Find the flow to each node

df_scenario2_part1_end_node_flow = df_scenario2_part1[['End Node', 'Flow']]
df_scenario2_part1_end_node_flow = df_scenario2_part1_end_node_flow.groupby(['End Node']).sum()
df_scenario2_part1_end_node_flow.reset_index(inplace = True)
df_scenario2_part1_end_node_flow

,End Node,Flow
0,1,0
1,2,955
2,3,1416
3,4,0
4,5,0
5,6,14064
6,7,3091
7,8,8751
8,9,9199
9,10,0


In [101]:
# Combine flow to/from each node

df_scenario2_part1_net_flow = (pd.merge(df_scenario2_part1_start_node_flow, df_scenario2_part1_end_node_flow, 
                               how = 'outer', left_on = 'Start Node', right_on = 'End Node'))
df_scenario2_part1_net_flow = df_scenario2_part1_net_flow.fillna(0)
df_scenario2_part1_net_flow['Node ID'] = df_scenario2_part1_net_flow[['Start Node', 'End Node']].max(axis = 1)
df_scenario2_part1_net_flow

,Start Node,Flow_x,End Node,Flow_y,Node ID
0,2.0,2476.0,2.0,955.0,2.0
1,3.0,2030.0,3.0,1416.0,3.0
2,4.0,955.0,4.0,0.0,4.0
3,5.0,735.0,5.0,0.0,5.0
4,7.0,3889.0,7.0,3091.0,7.0
5,9.0,11143.0,9.0,9199.0,9.0
6,10.0,393.0,10.0,0.0,10.0
7,11.0,3443.0,11.0,2476.0,11.0
8,12.0,0.0,12.0,7134.0,12.0
9,14.0,1416.0,14.0,0.0,14.0


In [102]:
# Calculate net flow from each node

df_scenario2_part1_net_flow['Net Flow - Scenario 2 part 1'] = (df_scenario2_part1_net_flow['Flow_x'] - 
                                                               df_scenario2_part1_net_flow['Flow_y'])
df_scenario2_part1_net_flow = df_scenario2_part1_net_flow[['Node ID', 'Net Flow - Scenario 2 part 1']]
df_scenario2_part1_net_flow

,Node ID,Net Flow - Scenario 2 part 1
0,2.0,1521.0
1,3.0,614.0
2,4.0,955.0
3,5.0,735.0
4,7.0,798.0
5,9.0,1944.0
6,10.0,393.0
7,11.0,967.0
8,12.0,-7134.0
9,14.0,1416.0


In [103]:
# Add columns in df_nodes_final and update values

df_nodes_final = pd.merge(df_nodes_final, df_scenario2_part1_net_flow, how = 'outer', on = 'Node ID')
df_nodes_final['Remaining Storage - Scenario 2 part 1'] = (df_nodes_final['Current Storage'] - 
                                                    df_nodes_final['Net Flow - Scenario 2 part 1'])
df_nodes_final.loc[df_nodes_final['Remaining Storage - Scenario 2 part 1'] < 0, 'Remaining Storage - Scenario 2 part 1'] = 0
df_nodes_final.loc[df_nodes_final['Demand'] > 0, 'Remaining Storage - Scenario 2 part 1'] = 0
df_nodes_final.loc[df_nodes_final['Demand'] > 0, 'Remaining Storage Capacity - Scenario 2 part 1'] = 0
df_nodes_final['Remaining Storage Capacity - Scenario 2 part 1'] = (df_nodes_final['Storage Capacity'] - 
                                                             df_nodes_final['Remaining Storage - Scenario 2 part 1'])
df_nodes_final['New Supplies - Scenario 2'] = (df_nodes_final['Supply'] - 
                                               df_nodes_final['Net Flow - Scenario 2 part 1'])
df_nodes_final.loc[df_nodes_final['New Supplies - Scenario 2'] < 0, 'New Supplies - Scenario 2'] = 0
df_nodes_final.loc[df_nodes_final['Resupply'] == 'No', 'New Supplies - Scenario 2'] = 0
df_nodes_final['New Demand - Scenario 2'] = df_nodes_final['Remaining Storage Capacity - Scenario 2 part 1']
df_nodes_final['New Supply/Demand - Scenario 2'] = (df_nodes_final['New Supplies - Scenario 2'] - 
                                                    df_nodes_final['New Demand - Scenario 2'])
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Net Flow - Scenario 1 part 2,Total Flow - Scenario 1,Remaining Storage - Scenario 1,Flow - Scenario 2 part 1,Net Flow - Scenario 2 part 1,Remaining Storage - Scenario 2 part 1,Remaining Storage Capacity - Scenario 2 part 1,New Supplies - Scenario 2,New Demand - Scenario 2,New Supply/Demand - Scenario 2
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,...,0.0,0,0.0,0,0.0,0.0,0.00,0.0,0.00,0.00
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,0.0,5106,0.0,2476,1521.0,0.0,2301.50,0.0,2301.50,-2301.50
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,...,-2495.0,2030,2495.0,2030,614.0,0.0,2495.35,0.0,2495.35,-2495.35
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,...,0.0,3585,0.0,955,955.0,0.0,2636.00,0.0,2636.00,-2636.00
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,...,0.0,735,0.0,735,735.0,0.0,2736.95,0.0,2736.95,-2736.95
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,...,0.0,14400,0.0,14064,-14064.0,0.0,0.00,0.0,0.00,0.00
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,...,-2671.0,6720,2671.0,3889,798.0,0.0,2671.10,0.0,2671.10,-2671.10
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,...,0.0,10350,0.0,8751,-8751.0,0.0,0.00,0.0,0.00,0.00
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,...,0.0,15845,0.0,11143,1944.0,0.0,2505.00,0.0,2505.00,-2505.00
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,...,0.0,393,0.0,393,393.0,0.0,2401.20,0.0,2401.20,-2401.20


In [104]:
# Create df_new_supplies to capture supply from all new nodes (blue nodes part 2)

df_new_supplies = df_nodes_final.copy()
df_new_supplies = df_new_supplies[['Node ID', 'New Supply/Demand - Scenario 2']]
df_new_supplies['Node ID'] += nodes_len
df_new_supplies['New Supply/Demand - Scenario 2'] = 0
df_new_supplies

,Node ID,New Supply/Demand - Scenario 2
0,50,0
1,51,0
2,52,0
3,53,0
4,54,0
5,55,0
6,56,0
7,57,0
8,58,0
9,59,0


In [105]:
# Create df_new_supplies_all to find supply from all black (part 1) and blue (part 2) nodes

df_new_supplies_all = pd.concat([df_nodes_final, df_new_supplies])
df_new_supplies_all = df_new_supplies_all.reset_index(drop = True)
df_new_supplies_all

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Net Flow - Scenario 1 part 2,Total Flow - Scenario 1,Remaining Storage - Scenario 1,Flow - Scenario 2 part 1,Net Flow - Scenario 2 part 1,Remaining Storage - Scenario 2 part 1,Remaining Storage Capacity - Scenario 2 part 1,New Supplies - Scenario 2,New Demand - Scenario 2,New Supply/Demand - Scenario 2
0,1,Washington AFB,End,Use,8500.0,0.00,0.0,7650.0,0.0,No,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030.0,2301.50,0.0,0.0,1521.0,No,...,0.0,5106.0,0.0,2476.0,1521.0,0.0,2301.50,0.0,2301.50,-2301.50
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907.0,2495.35,0.0,0.0,614.0,No,...,-2495.0,2030.0,2495.0,2030.0,614.0,0.0,2495.35,0.0,2495.35,-2495.35
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720.0,2636.00,0.0,0.0,955.0,No,...,0.0,3585.0,0.0,955.0,955.0,0.0,2636.00,0.0,2636.00,-2636.00
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739.0,2736.95,0.0,0.0,735.0,No,...,0.0,735.0,0.0,735.0,735.0,0.0,2736.95,0.0,2736.95,-2736.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
94,95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
95,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
96,97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00


In [106]:
# Create empty lists to use in the second min cost flow calculation

new_supplies = []
new_id_supplies = []

In [107]:
# Append lists in the correct format

for i in range(len(df_new_supplies_all)):
    new_supplies.append(int(df_new_supplies_all.loc[i, 'New Supply/Demand - Scenario 2']))
    new_id_supplies.append(int(df_new_supplies_all.loc[i, 'Node ID']))

In [108]:
# Create df_new_inputs in order to create lists for second min cost flow calculation

list = ({'Start Nodes': start_nodes, 'End Nodes': end_nodes, 
         'Remaining Expected Capacity': remaining_expected_capacity, 'Risk': risk})
df_new_inputs = pd.DataFrame(list)
df_new_inputs

,Start Nodes,End Nodes,Remaining Expected Capacity,Risk
0,51,10,65627,219
1,51,11,61687,140
2,52,6,35745,450
3,53,2,52456,140
4,54,8,24702,219
...,...,...,...,...
167,45,94,4840,120
168,46,95,53017,90
169,47,96,1800,120
170,48,97,2160,120


In [109]:
# Create empty lists to use in the second min cost flow calculation

start_nodes_2 = []
end_nodes_2 = []
remaining_expected_capacity_2 = []
risk_2 = []

In [110]:
# Append lists in the correct format

for i in range(len(df_new_inputs)):
    start_nodes_2.append(int(df_new_inputs.loc[i, 'Start Nodes']))
    end_nodes_2.append(int(df_new_inputs.loc[i, 'End Nodes']))
    remaining_expected_capacity_2.append(int(float(df_new_inputs.loc[i, 'Remaining Expected Capacity'])))
    risk_2.append(int(float(df_new_inputs.loc[i, 'Risk'])))

In [111]:
# Scenario 2 - part 2: Max flow with min cost using expected capacity

flow = pywrapgraph.SimpleMinCostFlow()

# Add each arc
for i in range(len(start_nodes)):
    flow.AddArcWithCapacityAndUnitCost(start_nodes_2[i], end_nodes_2[i], remaining_expected_capacity_2[i], risk_2[i])
    
# Add node supplies
for i in range(len(supplies)):
    flow.SetNodeSupply(int(new_id_supplies[i]), new_supplies[i])
    
start_node_output = []
end_node_output = []
flow_output = []
capacity_output = []
risk_output = []

start_node_output_v2 = []
end_node_output_v2 = []
flow_output_v2 = []
capacity_output_v2 = []
risk_output_v2 = []

# Find the minimum cost flow
if flow.SolveMaxFlowWithMinCost(): 
    for i in range(flow.NumArcs()):
        if start_nodes[i] > nodes_len:
            start_node_output.append(flow.Tail(i))
            end_node_output.append(flow.Head(i))
            flow_output.append(flow.Flow(i))
            capacity_output.append(flow.Capacity(i))
            risk_calc = (flow.UnitCost(i)/1000)
            risk_output.append(risk_calc)
        else:
            start_node_output_v2.append(flow.Tail(i))
            end_node_output_v2.append(flow.Head(i))
            flow_output_v2.append(flow.Flow(i))
            capacity_output_v2.append(flow.Capacity(i))
            risk_calc = (flow.UnitCost(i)/1000)
            risk_output_v2.append(risk_calc)
    
    total_flow_output_scenario2_part2 = sum(flow_output) + sum(flow_output_v2)
    total_flow_output_scenario2 = total_flow_output_scenario2_part1 + total_flow_output_scenario2_part2
    accum_risk_scenario2_part2 = float(flow.OptimalCost()) / 1000
    accum_risk_scenario2 = accum_risk_scenario2_part1 + accum_risk_scenario2_part2
    accum_risk_percent_scenario2 = accum_risk_scenario2 / total_flow_output_scenario2

# Create df_scenario2_part2 that includes flow along "true" nodes (black edges part 2)
df_scenario2_part2 = (pd.DataFrame(zip(start_node_output, end_node_output, flow_output, capacity_output, risk_output), 
                             columns = ['Start Node', 'End Node', 'Flow', 'Capacity', 'Risk']))
df_scenario2_part2['Start Node'] = df_scenario2_part2['Start Node'] - nodes_len
df_scenario2_part2['Edge ID'] = df_scenario2_part2.index + 1
df_scenario2_part2

,Start Node,End Node,Flow,Capacity,Risk,Edge ID
0,2,10,0,65627,0.219,1
1,2,11,0,61687,0.140,2
2,3,6,0,35745,0.450,3
3,4,2,0,52456,0.140,4
4,5,8,0,24702,0.219,5
...,...,...,...,...,...,...
118,46,48,0,72395,0.249,119
119,46,48,0,46521,0.480,120
120,47,48,0,20357,0.140,121
121,47,48,0,65910,0.219,122


In [112]:
# Create df_scenario2_part2_v2 that includes flow along "fake" nodes (blue edges part 1)

df_scenario2_part2_v2 = (pd.DataFrame(zip(start_node_output_v2, end_node_output_v2, flow_output_v2, capacity_output_v2, 
                   risk_output_v2), columns = ['Start Node', 'End Node', 'Flow', 'Capacity', 'Risk']))
df_scenario2_part2_v2

,Start Node,End Node,Flow,Capacity,Risk
0,1,50,0,8500,0.120
1,2,51,0,43554,0.060
2,3,52,0,47877,0.060
3,4,53,0,51765,0.060
4,5,54,0,54004,0.060
5,6,55,0,1936,0.121
6,7,56,2495,49533,0.060
7,8,57,0,2749,0.120
8,9,58,0,38957,0.060
9,10,59,0,47631,0.060


In [113]:
# Add scenario 2 flow part 2 to df_nodes_final

df_nodes_final = (pd.merge(df_nodes_final, df_scenario2_part2_v2, how = 'left', left_on = 'Node ID', 
                           right_on = 'Start Node'))
df_nodes_final = df_nodes_final.drop(['Start Node', 'End Node', 'Capacity', 'Risk'], axis = 1)
df_nodes_final = df_nodes_final.rename(columns = {'Flow': 'Flow - Scenario 2 part 2'})
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Total Flow - Scenario 1,Remaining Storage - Scenario 1,Flow - Scenario 2 part 1,Net Flow - Scenario 2 part 1,Remaining Storage - Scenario 2 part 1,Remaining Storage Capacity - Scenario 2 part 1,New Supplies - Scenario 2,New Demand - Scenario 2,New Supply/Demand - Scenario 2,Flow - Scenario 2 part 2
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,...,0,0.0,0,0.0,0.0,0.00,0.0,0.00,0.00,0
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,5106,0.0,2476,1521.0,0.0,2301.50,0.0,2301.50,-2301.50,0
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,...,2030,2495.0,2030,614.0,0.0,2495.35,0.0,2495.35,-2495.35,0
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,...,3585,0.0,955,955.0,0.0,2636.00,0.0,2636.00,-2636.00,0
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,...,735,0.0,735,735.0,0.0,2736.95,0.0,2736.95,-2736.95,0
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,...,14400,0.0,14064,-14064.0,0.0,0.00,0.0,0.00,0.00,0
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,...,6720,2671.0,3889,798.0,0.0,2671.10,0.0,2671.10,-2671.10,2495
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,...,10350,0.0,8751,-8751.0,0.0,0.00,0.0,0.00,0.00,0
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,...,15845,0.0,11143,1944.0,0.0,2505.00,0.0,2505.00,-2505.00,0
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,...,393,0.0,393,393.0,0.0,2401.20,0.0,2401.20,-2401.20,0


In [114]:
# Create df_scenario2_part2_gephi from df_scenario2_part2 to format for csv export and gephi import

df_scenario2_part2_gephi = df_scenario2_part2[df_scenario2_part2['Flow'] > 0]
df_scenario2_part2_gephi = df_scenario2_part2_gephi[['Start Node', 'End Node', 'Flow']]
df_scenario2_part2_gephi['Start-End Node'] = (df_scenario2_part2_gephi['Start Node'].astype(str) + ' - ' + 
                                              df_scenario2_part2_gephi['End Node'].astype(str))
df_scenario2_part2_gephi

,Start Node,End Node,Flow,Start-End Node
5,7,3,2495,7 - 3
35,19,7,5166,19 - 7
36,19,14,2295,19 - 14
37,19,22,2533,19 - 22
38,20,19,12426,20 - 19
60,28,27,1743,28 - 27
68,30,21,2661,30 - 21
70,30,23,2377,30 - 23
71,30,28,2,30 - 28
74,31,28,4185,31 - 28


In [115]:
# Create df_scenario2_gephi to format for csv export and gephi import

df_scenario2_gephi = pd.merge(df_scenario2_part1_gephi, df_scenario2_part2_gephi, how = 'outer', on = 'Start-End Node')
df_scenario2_gephi = df_scenario2_gephi.fillna(0)
df_scenario2_gephi['Source'] = df_scenario2_gephi[['Start Node_x', 'Start Node_y']].max(axis = 1)
df_scenario2_gephi['Target'] = df_scenario2_gephi[['End Node_x', 'End Node_y']].max(axis = 1)
df_scenario2_gephi['Weight'] = df_scenario2_gephi['Flow_x'] + df_scenario2_gephi['Flow_y']
df_scenario2_gephi = df_scenario2_gephi[['Source', 'Target', 'Weight']]
df_scenario2_gephi['Type'] = 'Directed'
convert_dict = {'Source': int, 'Target': int, 'Weight': int}
df_scenario2_gephi = df_scenario2_gephi.astype(convert_dict)
df_scenario2_gephi

,Source,Target,Weight,Type
0,2,11,2476,Directed
1,3,6,2030,Directed
2,4,2,955,Directed
3,5,8,735,Directed
4,7,6,3889,Directed
5,9,8,7623,Directed
6,9,13,3520,Directed
7,10,8,393,Directed
8,11,12,3443,Directed
9,14,3,1416,Directed


In [116]:
# Find the flow from each node

df_scenario2_part2_start_node_flow = df_scenario2_part2[['Start Node', 'Flow']]
df_scenario2_part2_start_node_flow = df_scenario2_part2_start_node_flow.groupby(['Start Node']).sum()
df_scenario2_part2_start_node_flow.reset_index(inplace = True)
df_scenario2_part2_start_node_flow

,Start Node,Flow
0,2,0
1,3,0
2,4,0
3,5,0
4,7,2495
5,9,0
6,10,0
7,11,0
8,12,0
9,14,0


In [117]:
# Find the flow to each node

df_scenario2_part2_end_node_flow = df_scenario2_part2[['End Node', 'Flow']]
df_scenario2_part2_end_node_flow = df_scenario2_part2_end_node_flow.groupby(['End Node']).sum()
df_scenario2_part2_end_node_flow.reset_index(inplace = True)
df_scenario2_part2_end_node_flow

,End Node,Flow
0,1,0
1,2,0
2,3,2495
3,4,0
4,5,0
5,6,0
6,7,5166
7,8,0
8,9,0
9,10,0


In [118]:
# Combine flow to/from each node

df_scenario2_part2_net_flow = (pd.merge(df_scenario2_part2_start_node_flow, df_scenario2_part2_end_node_flow, 
                                how = 'outer', left_on = 'Start Node', right_on = 'End Node'))
df_scenario2_part2_net_flow = df_scenario2_part2_net_flow.fillna(0)
df_scenario2_part2_net_flow['Node ID'] = df_scenario2_part2_net_flow[['Start Node', 'End Node']].max(axis = 1)
df_scenario2_part2_net_flow

,Start Node,Flow_x,End Node,Flow_y,Node ID
0,2.0,0.0,2.0,0.0,2.0
1,3.0,0.0,3.0,2495.0,3.0
2,4.0,0.0,4.0,0.0,4.0
3,5.0,0.0,5.0,0.0,5.0
4,7.0,2495.0,7.0,5166.0,7.0
5,9.0,0.0,9.0,0.0,9.0
6,10.0,0.0,10.0,0.0,10.0
7,11.0,0.0,11.0,0.0,11.0
8,12.0,0.0,12.0,0.0,12.0
9,14.0,0.0,14.0,2295.0,14.0


In [119]:
# Calculate net flow from each node

df_scenario2_part2_net_flow['Net Flow - Scenario 2 part 2'] = (df_scenario2_part2_net_flow['Flow_x'] - 
                                                               df_scenario2_part2_net_flow['Flow_y'])
df_scenario2_part2_net_flow = df_scenario2_part2_net_flow[['Node ID', 'Net Flow - Scenario 2 part 2']]
df_scenario2_part2_net_flow

,Node ID,Net Flow - Scenario 2 part 2
0,2.0,0.0
1,3.0,-2495.0
2,4.0,0.0
3,5.0,0.0
4,7.0,-2671.0
5,9.0,0.0
6,10.0,0.0
7,11.0,0.0
8,12.0,0.0
9,14.0,-2295.0


In [120]:
# Add columns in df_nodes_final and update values

df_nodes_final = pd.merge(df_nodes_final, df_scenario2_part2_net_flow, how = 'outer', on = 'Node ID')
df_nodes_final['Total Flow - Scenario 2'] = (df_nodes_final['Flow - Scenario 2 part 1'] + 
                                             df_nodes_final['Flow - Scenario 2 part 2'])
df_nodes_final['Remaining Storage - Scenario 2'] = (df_nodes_final['Remaining Storage - Scenario 2 part 1'] - 
                                                    df_nodes_final['Net Flow - Scenario 2 part 2'])
df_nodes_final.loc[df_nodes_final['Storage Capacity'] == 0, 'Remaining Storage - Scenario 2'] = 0
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Net Flow - Scenario 2 part 1,Remaining Storage - Scenario 2 part 1,Remaining Storage Capacity - Scenario 2 part 1,New Supplies - Scenario 2,New Demand - Scenario 2,New Supply/Demand - Scenario 2,Flow - Scenario 2 part 2,Net Flow - Scenario 2 part 2,Total Flow - Scenario 2,Remaining Storage - Scenario 2
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,...,0.0,0.0,0.00,0.0,0.00,0.00,0,0.0,0,0.0
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,1521.0,0.0,2301.50,0.0,2301.50,-2301.50,0,0.0,2476,0.0
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,...,614.0,0.0,2495.35,0.0,2495.35,-2495.35,0,-2495.0,2030,2495.0
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,...,955.0,0.0,2636.00,0.0,2636.00,-2636.00,0,0.0,955,0.0
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,...,735.0,0.0,2736.95,0.0,2736.95,-2736.95,0,0.0,735,0.0
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,...,-14064.0,0.0,0.00,0.0,0.00,0.00,0,0.0,14064,0.0
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,...,798.0,0.0,2671.10,0.0,2671.10,-2671.10,2495,-2671.0,6384,2671.0
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,...,-8751.0,0.0,0.00,0.0,0.00,0.00,0,0.0,8751,0.0
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,...,1944.0,0.0,2505.00,0.0,2505.00,-2505.00,0,0.0,11143,0.0
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,...,393.0,0.0,2401.20,0.0,2401.20,-2401.20,0,0.0,393,0.0


In [121]:
# Remove intermediary columns from df_nodes_final

df_nodes_final = df_nodes_final.drop(['Remaining Storage - Baseline part 1', 
                'Remaining Storage Capacity - Baseline part 1', 'New Supplies - Baseline', 'New Demand - Baseline', 
                'New Supply/Demand - Baseline', 'Remaining Storage - Scenario 1 part 1', 
                'Remaining Storage Capacity - Scenario 1 part 1', 'New Supplies - Scenario 1', 
                'New Demand - Scenario 1', 'New Supply/Demand - Scenario 1', 'Remaining Storage - Scenario 2 part 1', 
                'Remaining Storage Capacity - Scenario 2 part 1', 'New Supplies - Scenario 2', 
                'New Demand - Scenario 2', 'New Supply/Demand - Scenario 2'], axis = 1)
df_nodes_final

,Node ID,Node Name,Description,Type,Throughput,Storage Capacity,Supply,Demand,Current Storage,Resupply,...,Flow - Scenario 1 part 2,Net Flow - Scenario 1 part 2,Total Flow - Scenario 1,Remaining Storage - Scenario 1,Flow - Scenario 2 part 1,Net Flow - Scenario 2 part 1,Flow - Scenario 2 part 2,Net Flow - Scenario 2 part 2,Total Flow - Scenario 2,Remaining Storage - Scenario 2
0,1,Washington AFB,End,Use,8500,0.00,0,7650,0,No,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
1,2,Oregon bulk storage 1,Intermediate,Bulk Storage,46030,2301.50,0,0,1521,No,...,0,0.0,5106,0.0,2476,1521.0,0,0.0,2476,0.0
2,3,Oregon bulk storage 2,Intermediate,Bulk Storage,49907,2495.35,0,0,614,No,...,0,-2495.0,2030,2495.0,2030,614.0,0,-2495.0,2030,2495.0
3,4,Idaho bulk storage,Intermediate,Bulk Storage,52720,2636.00,0,0,955,No,...,0,0.0,3585,0.0,955,955.0,0,0.0,955,0.0
4,5,Montana bulk storage,Intermediate,Bulk Storage,54739,2736.95,0,0,735,No,...,0,0.0,735,0.0,735,735.0,0,0.0,735,0.0
5,6,North Dakota AFB,End,Use,16000,0.00,0,14400,0,No,...,0,0.0,14400,0.0,14064,-14064.0,0,0.0,14064,0.0
6,7,Wyoming bulk storage,Intermediate,Bulk Storage,53422,2671.10,0,0,798,No,...,2495,-2671.0,6720,2671.0,3889,798.0,2495,-2671.0,6384,2671.0
7,8,Idaho AFB,End,Use,11500,0.00,0,10350,0,No,...,0,0.0,10350,0.0,8751,-8751.0,0,0.0,8751,0.0
8,9,Utah bulk storage,Intermediate,Bulk Storage,50100,2505.00,0,0,1944,No,...,0,0.0,15845,0.0,11143,1944.0,0,0.0,11143,0.0
9,10,Nevada bulk storage 1,Intermediate,Bulk Storage,48024,2401.20,0,0,393,No,...,0,0.0,393,0.0,393,393.0,0,0.0,393,0.0


In [122]:
# Export node and edge files to csv for gephi import

df_baseline_gephi.to_csv('Supply_Chain_Model_Flow_Edges_Baseline.csv', index = False)
df_scenario1_gephi.to_csv('Supply_Chain_Model_Flow_Edges_Scenario1_Actual_Capacity.csv', index = False)
df_scenario2_gephi.to_csv('Supply_Chain_Model_Flow_Edges_Scenario2_Expected_Capacity.csv', index = False)
df_nodes_gephi.to_csv('Supply_Chain_Model_Nodes.csv', index = False)
df_nodes_gephi_adj.to_csv('Supply_Chain_Model_Nodes_obfuscated.csv', index = False)
df_edges_gephi.to_csv('Supply_Chain_Model_All_Edges.csv', index = False)

In [123]:
# Re-number start nodes in df_edges_risks to merge with 3 scenario dfs

df_edges_risks = df_edges_risks[df_edges_risks['Start Node'] > nodes_len]
df_edges_risks['Start Node'] = df_edges_risks['Start Node'] - nodes_len
df_edges_risks

<ipython-input-123-db9f8d3da2d9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edges_risks['Start Node'] = df_edges_risks['Start Node'] - nodes_len


,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Risks - Type,Risks - List,Risks - Location,Total Risks
0,1,2,10,Pipeline 1,blue,Pipeline,65627,0.22,0.0,0.000000e+00,0.22
1,2,2,11,Railroad 1,green,Railroad,64163,0.14,0.0,0.000000e+00,0.14
2,3,3,6,Truck 1,red,Truck,37775,0.45,0.0,8.673617e-19,0.45
3,4,4,2,Railroad 2,green,Railroad,53411,0.14,0.0,0.000000e+00,0.14
4,5,5,8,Pipeline 2,blue,Pipeline,25437,0.22,0.0,0.000000e+00,0.22
...,...,...,...,...,...,...,...,...,...,...,...
118,119,46,48,Pipeline 45,blue,Pipeline,72395,0.22,0.0,3.000000e-02,0.25
119,120,46,48,Truck 25,red,Truck,46521,0.45,0.0,3.000000e-02,0.48
120,121,47,48,Railroad 44,green,Railroad,20357,0.14,0.0,0.000000e+00,0.14
121,122,47,48,Pipeline 46,blue,Pipeline,65910,0.22,0.0,0.000000e+00,0.22


In [124]:
# Add baseline flow to df_edges_final

df_edges_final = pd.merge(df_edges_risks, df_baseline_part1, how = 'left', on = 'Edge ID')
df_edges_final = pd.merge(df_edges_final, df_baseline_part2, how = 'left', on = 'Edge ID')
df_edges_final = (df_edges_final[['Edge ID', 'Start Node_x', 'End Node_x', 'Edge Name', 'Description', 'Type', 
                                  'Capacity_x', 'Total Risks', 'Flow_x', 'Flow_y']])
df_edges_final = (df_edges_final.rename(columns = {'Flow_x': 'Flow - Baseline part 1', 
                                                   'Flow_y': 'Flow - Baseline part 2'}))
df_edges_final['Total Flow - Baseline'] = (df_edges_final['Flow - Baseline part 1'] + 
                                           df_edges_final['Flow - Baseline part 2'])
df_edges_final.columns = [x.replace('_x', '') for x in df_edges_final.columns]
df_edges_final

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Total Risks,Flow - Baseline part 1,Flow - Baseline part 2,Total Flow - Baseline
0,1,2,10,Pipeline 1,blue,Pipeline,65627,0.22,3585,0,3585
1,2,2,11,Railroad 1,green,Railroad,64163,0.14,1521,0,1521
2,3,3,6,Truck 1,red,Truck,37775,0.45,14303,0,14303
3,4,4,2,Railroad 2,green,Railroad,53411,0.14,3585,0,3585
4,5,5,8,Pipeline 2,blue,Pipeline,25437,0.22,10350,0,10350
...,...,...,...,...,...,...,...,...,...,...,...
118,119,46,48,Pipeline 45,blue,Pipeline,72395,0.25,0,0,0
119,120,46,48,Truck 25,red,Truck,46521,0.48,0,0,0
120,121,47,48,Railroad 44,green,Railroad,20357,0.14,1500,0,1500
121,122,47,48,Pipeline 46,blue,Pipeline,65910,0.22,0,0,0


In [125]:
# Add scenario 1 flow to df_edges_final

df_edges_final = pd.merge(df_edges_final, df_scenario1_part1, how = 'left', on = 'Edge ID')
df_edges_final = pd.merge(df_edges_final, df_scenario1_part2, how = 'left', on = 'Edge ID')
df_edges_final = (df_edges_final.drop(['Start Node_y', 'End Node_y', 'Capacity_y', 'Risk_x', 'Start Node', 'End Node', 
                                       'Capacity', 'Risk_y'], axis = 1))
df_edges_final = (df_edges_final.rename(columns = {'Flow_x': 'Flow - Scenario 1 part 1',
                                                   'Flow_y': 'Flow - Scenario 1 part 2'}))
df_edges_final['Total Flow - Scenario 1'] = (df_edges_final['Flow - Scenario 1 part 1'] + 
                                             df_edges_final['Flow - Scenario 1 part 2'])
df_edges_final.columns = [x.replace('_x', '') for x in df_edges_final.columns]
df_edges_final

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Total Risks,Flow - Baseline part 1,Flow - Baseline part 2,Total Flow - Baseline,Flow - Scenario 1 part 1,Flow - Scenario 1 part 2,Total Flow - Scenario 1
0,1,2,10,Pipeline 1,blue,Pipeline,65627,0.22,3585,0,3585,0,0,0
1,2,2,11,Railroad 1,green,Railroad,64163,0.14,1521,0,1521,5106,0,5106
2,3,3,6,Truck 1,red,Truck,37775,0.45,14303,0,14303,2030,0,2030
3,4,4,2,Railroad 2,green,Railroad,53411,0.14,3585,0,3585,3585,0,3585
4,5,5,8,Pipeline 2,blue,Pipeline,25437,0.22,10350,0,10350,735,0,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,119,46,48,Pipeline 45,blue,Pipeline,72395,0.25,0,0,0,0,0,0
119,120,46,48,Truck 25,red,Truck,46521,0.48,0,0,0,0,0,0
120,121,47,48,Railroad 44,green,Railroad,20357,0.14,1500,0,1500,0,0,0
121,122,47,48,Pipeline 46,blue,Pipeline,65910,0.22,0,0,0,0,0,0


In [126]:
# Add scenario 2 flow to df_edges_final

df_edges_final = pd.merge(df_edges_final, df_scenario2_part1, how = 'left', on = 'Edge ID')
df_edges_final = pd.merge(df_edges_final, df_scenario2_part2, how = 'left', on = 'Edge ID')
df_edges_final = (df_edges_final.drop(['Start Node_y', 'End Node_y', 'Capacity_y', 'Risk_x', 'Start Node', 'End Node', 
                                       'Capacity', 'Risk_y'], axis = 1))
df_edges_final = (df_edges_final.rename(columns = {'Flow_x': 'Flow - Scenario 2 part 1',
                                                   'Flow_y': 'Flow - Scenario 2 part 2'}))
df_edges_final['Total Flow - Scenario 2'] = (df_edges_final['Flow - Scenario 2 part 1'] + 
                                             df_edges_final['Flow - Scenario 2 part 2'])
df_edges_final.columns = [x.replace('_x', '') for x in df_edges_final.columns]
df_edges_final

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Total Risks,Flow - Baseline part 1,Flow - Baseline part 2,Total Flow - Baseline,Flow - Scenario 1 part 1,Flow - Scenario 1 part 2,Total Flow - Scenario 1,Flow - Scenario 2 part 1,Flow - Scenario 2 part 2,Total Flow - Scenario 2
0,1,2,10,Pipeline 1,blue,Pipeline,65627,0.22,3585,0,3585,0,0,0,0,0,0
1,2,2,11,Railroad 1,green,Railroad,64163,0.14,1521,0,1521,5106,0,5106,2476,0,2476
2,3,3,6,Truck 1,red,Truck,37775,0.45,14303,0,14303,2030,0,2030,2030,0,2030
3,4,4,2,Railroad 2,green,Railroad,53411,0.14,3585,0,3585,3585,0,3585,955,0,955
4,5,5,8,Pipeline 2,blue,Pipeline,25437,0.22,10350,0,10350,735,0,735,735,0,735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,119,46,48,Pipeline 45,blue,Pipeline,72395,0.25,0,0,0,0,0,0,0,0,0
119,120,46,48,Truck 25,red,Truck,46521,0.48,0,0,0,0,0,0,0,0,0
120,121,47,48,Railroad 44,green,Railroad,20357,0.14,1500,0,1500,0,0,0,0,0,0
121,122,47,48,Pipeline 46,blue,Pipeline,65910,0.22,0,0,0,0,0,0,0,0,0


In [127]:
# Add column 'Required Flow - Scenario 2' to df_edges_final with temporary values of 0

df_edges_final['Required Flow - Scenario 2'] = 0
df_edges_final

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Total Risks,Flow - Baseline part 1,Flow - Baseline part 2,Total Flow - Baseline,Flow - Scenario 1 part 1,Flow - Scenario 1 part 2,Total Flow - Scenario 1,Flow - Scenario 2 part 1,Flow - Scenario 2 part 2,Total Flow - Scenario 2,Required Flow - Scenario 2
0,1,2,10,Pipeline 1,blue,Pipeline,65627,0.22,3585,0,3585,0,0,0,0,0,0,0
1,2,2,11,Railroad 1,green,Railroad,64163,0.14,1521,0,1521,5106,0,5106,2476,0,2476,0
2,3,3,6,Truck 1,red,Truck,37775,0.45,14303,0,14303,2030,0,2030,2030,0,2030,0
3,4,4,2,Railroad 2,green,Railroad,53411,0.14,3585,0,3585,3585,0,3585,955,0,955,0
4,5,5,8,Pipeline 2,blue,Pipeline,25437,0.22,10350,0,10350,735,0,735,735,0,735,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,119,46,48,Pipeline 45,blue,Pipeline,72395,0.25,0,0,0,0,0,0,0,0,0,0
119,120,46,48,Truck 25,red,Truck,46521,0.48,0,0,0,0,0,0,0,0,0,0
120,121,47,48,Railroad 44,green,Railroad,20357,0.14,1500,0,1500,0,0,0,0,0,0,0
121,122,47,48,Pipeline 46,blue,Pipeline,65910,0.22,0,0,0,0,0,0,0,0,0,0


In [128]:
# Calculate values in 'Required Flow - Scenario 2'

for i in df_edges_final['Required Flow - Scenario 2']:
    if not df_edges_final.loc[i, 'Total Risks'] == 1:
        df_edges_final['Required Flow - Scenario 2'] = (df_edges_final['Total Flow - Scenario 2'] / 
                                                       (1 - df_edges_final['Total Risks']))
    else:
        df_edges_final['Required Flow - Scenario 2'] == 0
df_edges_final

,Edge ID,Start Node,End Node,Edge Name,Description,Type,Capacity,Total Risks,Flow - Baseline part 1,Flow - Baseline part 2,Total Flow - Baseline,Flow - Scenario 1 part 1,Flow - Scenario 1 part 2,Total Flow - Scenario 1,Flow - Scenario 2 part 1,Flow - Scenario 2 part 2,Total Flow - Scenario 2,Required Flow - Scenario 2
0,1,2,10,Pipeline 1,blue,Pipeline,65627,0.22,3585,0,3585,0,0,0,0,0,0,0.000000
1,2,2,11,Railroad 1,green,Railroad,64163,0.14,1521,0,1521,5106,0,5106,2476,0,2476,2879.069767
2,3,3,6,Truck 1,red,Truck,37775,0.45,14303,0,14303,2030,0,2030,2030,0,2030,3690.909091
3,4,4,2,Railroad 2,green,Railroad,53411,0.14,3585,0,3585,3585,0,3585,955,0,955,1110.465116
4,5,5,8,Pipeline 2,blue,Pipeline,25437,0.22,10350,0,10350,735,0,735,735,0,735,942.307692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,119,46,48,Pipeline 45,blue,Pipeline,72395,0.25,0,0,0,0,0,0,0,0,0,0.000000
119,120,46,48,Truck 25,red,Truck,46521,0.48,0,0,0,0,0,0,0,0,0,0.000000
120,121,47,48,Railroad 44,green,Railroad,20357,0.14,1500,0,1500,0,0,0,0,0,0,0.000000
121,122,47,48,Pipeline 46,blue,Pipeline,65910,0.22,0,0,0,0,0,0,0,0,0,0.000000


In [129]:
# Create lit 'summary_stats' that includes accumulated risk values from the 2 non-baseline scenarios

summary_stats = ([accum_risk_scenario1, accum_risk_percent_scenario1, accum_risk_scenario2, 
                  accum_risk_percent_scenario2])
summary_stats

[121411.228, 0.18382299512929254, 109977.764, 0.1934923528676966]

In [130]:
# Create list 'summary_stats_names'

summary_stats_names = (['Scenario 1 - Risk-based routing actual capacity', 
                        'Scenario 1 - Risk-based routing actual capacity',
                        'Scenario 2 - Risk-based routing expected capacity', 
                        'Scenario 2 - Risk-based routing expected capacity'])
summary_stats_names

['Scenario 1 - Risk-based routing actual capacity',
 'Scenario 1 - Risk-based routing actual capacity',
 'Scenario 2 - Risk-based routing expected capacity',
 'Scenario 2 - Risk-based routing expected capacity']

In [131]:
# Create list 'summary_stats_description'

summary_stats_description = (['Accumulated Risk', 'Accumulated Risk Percent',
                              'Accumulated Risk', 'Accumulated Risk Percent',])
summary_stats_description

['Accumulated Risk',
 'Accumulated Risk Percent',
 'Accumulated Risk',
 'Accumulated Risk Percent']

In [132]:
# Create 'df_summary' dataframe combining lists 'summary_stats_names', 'summary_stats_description', 'summary_stats'

df_summary = (pd.DataFrame(zip(summary_stats_names, summary_stats_description, summary_stats), 
                          columns = ['Scenario Description', 'Statistic', 'Value']))
df_summary

,Scenario Description,Statistic,Value
0,Scenario 1 - Risk-based routing actual capacity,Accumulated Risk,121411.228000
1,Scenario 1 - Risk-based routing actual capacity,Accumulated Risk Percent,0.183823
2,Scenario 2 - Risk-based routing expected capacity,Accumulated Risk,109977.764000
3,Scenario 2 - Risk-based routing expected capacity,Accumulated Risk Percent,0.193492


In [133]:
# Export 5 original sheets & summary sheet to Excel with 3 scenario flow columns added to 'Edges' sheet

writer = pd.ExcelWriter('OUT_Supply_Chain_Model.xlsx', engine = 'xlsxwriter')
df_summary.to_excel(writer, sheet_name = 'Summary', index = False)
df_nodes_final.to_excel(writer, sheet_name = 'Nodes', index = False)
df_edges_final.to_excel(writer, sheet_name = 'Edges', index = False)
df_risks_location_orig.to_excel(writer, sheet_name = 'Risks - Location', index = False)
df_risks_type_orig.to_excel(writer, sheet_name = 'Risks - Type', index = False)
df_risks_list_orig.to_excel(writer, sheet_name = 'Risks - List', index = False)
writer.save()